# COM3502-4502-6502 Speech Processing - Python Programming Assignment

## General Information

This programming assignment is worth $55$% of the overall course mark.

You are free to complete this assignment in your own time. However, feedback, advice and guidance are available during the lab classes and via the discussion board on Blackboard. 

Note: Via these channels, we try to help you as much as possible, but will not debug your code or provide solutions to the assignment itself.

Note: It will take some time to complete this assignment, so plan your work accordingly over the coming weeks. Read these instructions carefully.

Note: Please be aware that students registered on COM4502 and COM6502 have **additional tasks** to perform. These are marked ‘COM4502-6502 Only’.

Note: You should always ensure that your results (e.g. in terms of plots you create) are clear to understand and leave no room for misinterpretation. This can often be easily achieved by adding proper $x$- and $y$-axis labels, titles, legends etc. Where results are not clear to interpret, this might result in missed points. 


## Student Data

Student Family Name: <span style="font-weight:bold;color:orange">**Zheng**</span>

Student Given Name(s): <span style="font-weight:bold;color:orange">**Fengyuan**</span>

Date of submission: <span style="font-weight:bold;color:orange">**Friday 19th December 2024**</span>

## Copyright

This programming assignment is part of the lecture COM[3502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level3/com3502.html "Open web page for COM3502 module")-[4502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/level4/com4502.html "Open web page for COM4502 module")-[6502](http://www.dcs.shef.ac.uk/intranet/teaching/public/modules/msc/com6502.html "Open web page for COM4502 module") Speech Processing at the [University of Sheffield](https://www.sheffield.ac.uk/ "Open web page of The University of Sheffield"), Dept. of [Computer Science](https://www.sheffield.ac.uk/dcs "Open web page of Department of Computer Science"), University of Sheffield.

This notebook is licensed as an assignment to be used during the lecture COM3502-4502-6502 Speech Processing at the University of Sheffield. Any further use is only permitted if agreed with the [module lead](mailto:s.goetze@sheffield.ac.uk).

It should be a matter of course that rules of [unfair means](https://www.sheffield.ac.uk/apse/apo/quality/assessment/unfair) apply and the assignment is not to be shared with or made available to other persons besides those participating in the module during the same academic year. This includes publishing on web pages etc. All questions can be asked during the lab classes or using the Blackboard Discussion board.


## Hand-In Procedure and Deadline

Once you have completed the assignment you should submit a `.zip` file (via Blackboard) containing your solution (as a file named `YourFamilyName.ipynb`) and possibly other sources linked in your Jupyter Notebook. Also, the `.zip` filename should be of the form `YourFamilyName.zip`. Please also ensure that your name is entered correctly in the section above.

Standard departmental penalties apply for [late hand-in](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) and [plagiarism](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means).

The **deadline** for handing-in this assignment (via Blackboard) is 
<span style="font-weight:bold;color:red">**Friday 19th December 2024**</span>

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task 0:**
    
Ensure that you data is correctly entered in the section at the top of this sheet and that the filename is in the form `YourFamilyName.ipynb`. 
    
</div>

## Libraries

You should be familiar with the use of the following Python libraries from the lab. You should not need to use additional ones. You are allowed to use additional libraries if necessary for your code. If they need to be installed by `!pip install <libraryname>` or `!conda install <libraryname>`, please indicate this as a comment in your code. You should not make use of libraries that can't be installed by either `!pip install` or `!conda install`. You must ensure that your Notebook runs "out of the box". You can test this on the Computer Lab machines in the Diamond if you are unsure and using your own computer.

In [ ]:
#Let's do some necessary and nice-to-have imports
%matplotlib inline
import matplotlib.pyplot as plt    # plotting
#import seaborn as sns; sns.set()  # styling
import numpy as np                 # math
import soundfile as sf             # to load files
from IPython import display as ipd # for sound playback
from scipy import signal           # filter designs (if not already imported)

# Download, load, and analyse audio

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T1:** 
    
* Load a wave file containing speech. You can find a file at <a href="https://staffwww.dcs.shef.ac.uk/people/S.Goetze/sound/speech.wav"> https://staffwww.dcs.shef.ac.uk/people/S.Goetze/sound/speech.wav</a> and should be able to download this. You can also use your own WAVE files if you prefer this. If you want to record WAVE files and are using your own computer, the program [Audacity](https://www.audacityteam.org/download/) is one possibility to [record WAVE files](https://manual.audacityteam.org/man/basic_recording_editing_and_exporting.html).
    
* Visualise the signal in time domain, in the spectral domain (as spectrum) and as a time-frequency representation (spectrogram). Please ensure proper axis labels for all your plot in this assignment.

* Playback the signal.
    
</div>

In [ ]:
# Load the wave file
file_name = 'speech.wav'
!curl https://staffwww.dcs.shef.ac.uk/people/S.Goetze/sound/{file_name} -o {file_name}
speech_signal, speech_signal_fs = sf.read(file_name)
print(speech_signal.shape)
print(speech_signal_fs)

In [ ]:
# Visualise the signal in time domain
speech_signal_time = len(speech_signal)/speech_signal_fs
t = np.linspace(0, speech_signal_time, len(speech_signal), endpoint=False)
plt.figure(figsize=(12,6))
plt.plot(t, speech_signal)
plt.xlabel('Time in second')
plt.ylabel('Amplitude')
plt.title('Time domain representation of the speech signal')

In [ ]:
# Visualise the signal in frequency domain, using FFT is faster than DFT
signal_length = len(speech_signal)  # the length of the signal is 928086 
# we need to find the L_DFT that is the exponential of 2 and should be larger than the length of signal
L_DFT = 2**int(np.ceil(np.log2(signal_length)))  #2**20

# perform DFT using the FFT algorithm
spectrum = np.fft.fft(speech_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/speech_signal_fs) # the order of frequency is correspond to the output of FFT
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

plt.figure(figsize=(12, 6))
# plot the full frequency domain of the speech signal
plt.subplot(1, 2, 1)
plt.plot(frequency, magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Full frequency domain representation of the speech signal')

# plot the positive frequency domain of the speech signal
plt.subplot(1, 2, 2)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Positive frequency domain representation of the speech signal')

# find the highest occuring frequency
max_magnitude_index = np.argmax(positive_magnitude_spectrum)
max_frequency = positive_frequency[max_magnitude_index]
print(max_frequency)

In [ ]:
# Visualise the speech signal in time-frequency 
# small NFFT length leads to high time resolution and low frequency resolution
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.specgram(speech_signal, Fs=speech_signal_fs, NFFT=512, noverlap=256, cmap='viridis', scale='dB')
plt.title('spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')

# large NFFT length leads to low time resolution and high frequency resolution
plt.subplot(1, 2, 2)
plt.specgram(speech_signal, Fs=speech_signal_fs, NFFT=8192, noverlap=4096, cmap='viridis', scale='dB')
plt.title('spectrogram with DFT length of 8192')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')        

In [ ]:
# playback the speech audio
ipd.Audio(speech_signal, rate=speech_signal_fs)

<br>
<div style="border: 2px solid #999; padding: 10px; background: #9696ff;">
    
**Question Q1:** 

* Determine the sampling frequency $f_s$ in Hz and the length of the signal in seconds. What is the sampling interval $T_s$ of your signal? What is the highest occurring frequency?
    
Note: You can either give your answer in the form of a code block (e.g. by using the `print()` functions) or as text. For the latter, change the [type of the next cell](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#structure-of-a-notebook-document) from `code` to `markdown` or use the yellow example text below.
    
</div>

<span style="font-weight:bold;color:orange">
    Sampling frequency $f_s$: Read the speech signal by using soundfile, the sampling frequency $f_s$ is 44100 HZ. <br><br>
    Length of the signal (in seconds): The length of the signal can be calculated using:  <br><br>
    $$
\text{Signal Length (Seconds)} = \frac{\text{Number of Samples}}{\text{sampling frequency}}
    $$ <br><br>
    By calculate the formula, we can get signal length is 21.045s to 3 d.p. <br><br>
    Sampling interval $T_s$: The sampling interval can be calculated by taking the inverse of sampling frequency $f_s$, which is $2.268\times 10^{-5}$s to 3 d.p. <br><br>
    Highest occuring frequency: the highest frequecy in a signal is determined by the Nyquist freqeucy, which is half of the sampling frequency equal to 20250 HZ. The highest occuring frequency is 176.56 HZ.<br>
</span>

## Signal Analysis

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T2:** 
    
* Generate a synthetic audio signal consisting of three sinusoids with frequencies of $100$ Hz, $300$ Hz, and $500$ Hz including an initial phase which should be different from zero at least for one of the sinusoids. Assume a sampling rate of $8$ kHz and a duration of $2$ seconds. Write code to generate and plot the waveform of this signal. Compute and plot the magnitude spectrum of the signal.
    
* Use the Fast Fourier Transform (FFT) to calculate the spectrum, and display only the positive frequencies. Identify the main frequency components in the signal (using Python code) based on the magnitude spectrum and briefly explain your observations and identify the three main frequency peaks (as written answer below).

</div>

In [ ]:
# set the sampling rate to 8000 and the duration to 2 seconds
audio_signal_fs = 8000
audio_signal_time = 2
audio_signal_t = np.linspace(0, audio_signal_time, int(audio_signal_fs*audio_signal_time), endpoint=False) 

# for 100HZ with initial phase 0, 300HZ with initial phase π/4, 500HZ with initial phase π/2
frequencies = [100, 300, 500]
initial_phases = [0, np.pi/4, np.pi/2]
audio_signal = (
    np.sin(2 * np.pi * frequencies[0] * audio_signal_t + initial_phases[0])
    + np.sin(2 * np.pi * frequencies[1] * audio_signal_t + initial_phases[1])
    + np.sin(2 * np.pi * frequencies[2] * audio_signal_t + initial_phases[2])
)

# plot the waveform of synthetic audio signal
plt.figure(figsize=(12, 6))
plt.plot(audio_signal_t, audio_signal)
plt.title("Waveform of the Synthetic Audio Signal")
plt.xlabel("Time in seconds")
plt.ylabel("Amplitude")

In [ ]:
# to use the fast fourier transformation we need to find the LDFT
audio_signal_length = len(audio_signal) # the length of the signal is 16000
L_DFT = 2 ** int(np.ceil(np.log2(audio_signal_length)))  # 2**14
spectrum = np.fft.fft(audio_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequencies = np.fft.fftfreq(L_DFT, d=1/audio_signal_fs)
# By the theory of Nyquist, the positive frequency is from 0 to fs/2
positive_frequencies = frequencies[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

plt.figure(figsize=(12, 6))
# plot the full frequency domain of the synthetic signal
plt.subplot(1, 2, 1)
plt.plot(frequencies, magnitude_spectrum)
plt.title('Magnitude spectrum of the aduio signal')
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')

# plot the positive frequency domain of the synthetic signal
plt.subplot(1, 2, 2)
plt.plot(positive_frequencies, positive_magnitude_spectrum)
plt.title('Magnitude spectrum of the aduio signal with positive frequency')
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')

In [ ]:
top3_amplitude_index = np.argsort(positive_magnitude_spectrum)[-3:]
top3_frequency = positive_frequencies[top3_amplitude_index]
print(top3_frequency)

<span style="font-weight:bold;color:orange">According to the code results, the main frequency components in the signal are 299.8 Hz, 100.1 Hz, and 500 Hz. These values appear slightly shifted due to the use of the Discrete Fourier Transform (DFT) for transforming the time-domain signal to the frequency domain. The DFT operates on finite-length signals and has limited frequency resolution, which can cause small deviations (like the decimals observed here). <br><br>
From the magnitude spectrum of the audio signal, we can clearly observe three distinct peaks, each corresponding to a frequency component with a high amplitude. These peaks occur at the frequencies of the sinusoidal components used to construct the signal. The absence of significant amplitudes at other frequencies confirms that the signal is composed of exactly three sinusoids. The identified frequency peaks are 100 Hz, 300 Hz, and 500 Hz.</span>

# Piece-wise linear filtering in the time domain

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T3:** 
    
* Design a high-pass filter with a cut-off frequency of $\approx 500$ Hz using a filter design method of your choice. 
* Design a low-pass filter with a cut-off frequency of $\approx 500$ Hz.
* Design a band-stop filter with a cut-off frequencies of $\approx 300$ Hz and of $\approx 1.1$ kHz.
* Simulate the effect of a land-line telephone by eliminating all energy below $300$ Hz and above $3,400$ Hz.
* Visualise the transfer functions of the filters and the zero-pole plots.
* Apply the designed filters, compare filter input and output as a time-frequency visualisation and play back the filtered signal.
    
Note: Don't forget proper labeling/description of your figures to make clear what is what.
    
Note: In case you encounter stability problems, remember that we mentioned in the lecture, that filters can be designed as second-order-systems (SOS) which the design methods you are familiar with can realise.
</div>

In [ ]:
# create a signal to test different filters
target_rate = 8000
num_samples = int(len(speech_signal) * target_rate / speech_signal_fs)
resampled_signal = signal.resample(speech_signal, num_samples)
resampled_signal_time = len(resampled_signal)/target_rate
resampled_t = np.linspace(0, resampled_signal_time, len(resampled_signal), endpoint=False)

### High pass filter 

In [ ]:
# Design a high-pass filter with a cut-off frequency of 500HZ by a Butterworth filter
filter_sampling_rate = 8000
nyquist = 0.5 * filter_sampling_rate
cutoff_freq = 500
W_act = cutoff_freq / nyquist
W_bit = 0.025
Wp = W_act + W_bit  # passband edge frequency
Ws = W_act - W_bit  # stopband edge frequency
Rp_lin = 0.9  # allowed ripples in the pass band area
Rs_lin = 0.1  # allowed ripples in the stop band area

# draw the tolerance scheme
def plot_tolerance_scheme_high_pass(Wp, Ws, Rp_lin, Rs_lin):
    dh1x=[0,Wp];  dh1y=[0,0];            
    dh2x=[0,Ws];  dh2y=[Rs_lin,Rs_lin];
    dv2x=[Ws,Ws]; dv2y=[Rs_lin,1];   
    sh1x=[Ws,1];  sh1y=[1,1]; 
    sh2x=[Wp,1];  sh2y=[Rp_lin,Rp_lin]; 
    svx=[Wp,Wp];  svy=[0,Rp_lin]; 
    plt.plot(dh1x,dh1y,'k--',dh2x,dh2y,'k--',dv2x,dv2y,'k--',sh1x,sh1y,'k--',
             sh2x,sh2y,'k--',svx,svy,'k--')
    plt.xlabel('Frequency $\Omega/\pi$')
    plt.ylabel('Amplitude $|h(e^{j \Omega})|$')
plot_tolerance_scheme_high_pass(Wp, Ws, Rp_lin, Rs_lin)

Rp = -20 * np.log10(Rp_lin)
Rs = -20 * np.log10(Rs_lin)
N, Wn = signal.buttord(Wp, Ws, Rp, Rs)
print('The minimum possible filter order to fulfil the tolerance scheme is '+str(N)+'.')
print('The cut-off frequency which will be {:.2f}.'.format(Wn)) # format number - two digits after decimal pt.

In [ ]:
# Implement a transfer function visualization plot
def plot_transfer_function(b, a, filter_type):
    f, h = signal.freqz(b, a)
    omega = np.linspace(0, 1, len(f))
    plt.plot(omega, np.abs(h), lw=2, label='Butterworth '+str(filter_type)+' filter')
    plt.title('Butterworth ' + str(filter_type) + ' filter of order ' + str(N))

In [ ]:
# design high-pass filter of order N using butterworth method
b, a = signal.butter(N, Wn, 'high')
# plot the transfer function with tolerance scheme
plot_tolerance_scheme_high_pass(Wp, Ws, Rp_lin, Rs_lin)
plt.plot([Wn,Wn],[0,1],color='red',ls=':',label='cutoff frequency')
plot_transfer_function(b=b, a=a, filter_type='high pass')
plt.legend()
plt.show()

In [ ]:
def zplane(z, p, filter_type):
    "Plots zeros and poles in the complex z-plane"
    ax = plt.gca()

    ax.plot(np.real(z), np.imag(z), 'bo', fillstyle='none', ms=10)
    ax.plot(np.real(p), np.imag(p), 'rx', fillstyle='none', ms=10)
    unit_circle = plt.Circle((0, 0), radius=1, fill=False,
                             color='black', ls='--', alpha=0.9)
    ax.add_patch(unit_circle)

    plt.title('Poles and Zeros of ' +str(filter_type)+ ' filter')
    plt.xlabel('Re{$z$}')
    plt.ylabel('Im{$z$}')
    plt.axis('equal')
# plot zeros and poles in the z plane
zplane(np.roots(b), np.roots(a), filter_type='high pass')

In [ ]:
high_pass_filtered_signal = signal.filtfilt(b, a, resampled_signal)
plt.figure(figsize=(12, 6))
plt.plot(resampled_t, resampled_signal, color='b', label='resampled signal')
plt.plot(resampled_t, high_pass_filtered_signal, color='r', label='filtered signal')
plt.xlabel('Time in seconds')
plt.ylabel('Amplitude')
plt.title('Time domain representation of Resampled and Filtered Signal')
plt.legend()

In [ ]:
signal_length = len(resampled_signal)  
L_DFT = 2**int(np.ceil(np.log2(signal_length)))  

# perform DFT using the FFT algorithm
spectrum = np.fft.fft(resampled_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/target_rate) # the order of frequency is correspond to the output of FFT
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

high_pass_signal_length = len(high_pass_filtered_signal)
high_pass_L_DFT = 2**int(np.ceil(np.log2(high_pass_signal_length)))  
spectrum_filtered = np.fft.fft(high_pass_filtered_signal, high_pass_L_DFT)
magnitude_spectrum_filtered = np.abs(spectrum_filtered)
frequency_filtered = np.fft.fftfreq(high_pass_L_DFT, d=1/target_rate) # the order of frequency is correspond to the output of FFT
positive_frequency_filtered = frequency_filtered[:high_pass_L_DFT//2]
positive_magnitude_spectrum_filtered = magnitude_spectrum_filtered[:high_pass_L_DFT//2]

# plot the positive frequency domain of the speech signal
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.ylim(0, 500)
plt.title('Frequency domain representation of the resampled signal')

# plot the positive frequency domain of the filtered speech signal
plt.subplot(1, 2, 2)
plt.plot(positive_frequency_filtered, positive_magnitude_spectrum_filtered)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.ylim(0, 500)
plt.title('Frequency domain representation of the high pass filtered resampled signal')

In [ ]:
# plot the original and high pass filtered speech signal time-frequency diagram
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.specgram(resampled_signal, Fs=target_rate, NFFT=512, noverlap=256, cmap='viridis', scale='dB', vmin=-240, vmax=-40)
plt.title('Original spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')

plt.subplot(1, 2, 2)
plt.specgram(high_pass_filtered_signal, Fs=target_rate, NFFT=512, noverlap=256, cmap='viridis', scale='dB', vmin=-240, vmax=-40)
plt.title('High pass filtered spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')  

In [ ]:
ipd.Audio(resampled_signal, rate=target_rate)

In [ ]:
ipd.Audio(high_pass_filtered_signal, rate=target_rate)

### Low pass filter

In [ ]:
# Design a low-pass filter with a cut-off frequency of 500HZ by a Butterworth filter
filter_sampling_rate = 8000
nyquist = 0.5 * filter_sampling_rate
cutoff_freq = 500
W_act = cutoff_freq / nyquist
# we need two frequencies for the tolerance scheme, we chose one of them a bit smaller than W_act and the other a bit larger than o.5, we define 
# a bit is equal to 0.025
W_bit = 0.025
Wp = W_act - W_bit  # passband edge frequency
Ws = W_act + W_bit  # stopband edge frequency
Rp_lin = 0.9  # allowed ripples in the pass band area
Rs_lin = 0.1  # allowed ripples in the stop band area

# draw the tolerance scheme
def plot_tolerance_scheme_low_pass(Wp, Ws, Rp_lin, Rs_lin):
    dh1x=[0,Ws];  dh1y=[1,1];            
    dh2x=[0,Wp];  dh2y=[Rp_lin,Rp_lin];
    dv2x=[Wp,Wp]; dv2y=[0,Rp_lin];   
    sh1x=[Ws,1];  sh1y=[Rs_lin,Rs_lin]; 
    sh2x=[Wp,1];  sh2y=[0,0]; 
    svx=[Ws,Ws];  svy=[Rs_lin,1];  
    # plot the actual lines
    plt.plot(dh1x,dh1y,'k--',dh2x,dh2y,'k--',dv2x,dv2y,'k--',sh1x,sh1y,'k--',
             sh2x,sh2y,'k--',svx,svy,'k--')
    plt.xlabel('Frequency $\Omega/\pi$')
    plt.ylabel('Amplitude $|h(e^{j \Omega})|$')
plot_tolerance_scheme_low_pass(Wp,Ws,Rp_lin,Rs_lin)

Rp = -20 * np.log10(Rp_lin)
Rs = -20 * np.log10(Rs_lin)
N, Wn = signal.buttord(Wp, Ws, Rp, Rs)
print('The minimum possible filter order to fulfil the tolerance scheme is '+str(N)+'.')
print('The cut-off frequency which will be {:.2f}.'.format(Wn)) # format number - two digits after decimal pt.

In [ ]:
# design low-pass filter of order N using butterworth method
b, a = signal.butter(N, Wn, 'low')
# plot the transfer function with tolerance scheme
plot_tolerance_scheme_low_pass(Wp, Ws, Rp_lin, Rs_lin)
plt.plot([Wn,Wn],[0,1],color='red',ls=':',label='cutoff frequency')
plot_transfer_function(b=b, a=a, filter_type='low pass')
plt.legend()
plt.show()

In [ ]:
# plot zeros and poles in the z plane
zplane(np.roots(b), np.roots(a), filter_type='low pass')

In [ ]:
low_pass_filtered_signal = signal.filtfilt(b, a, resampled_signal)
plt.figure(figsize=(12, 6))
plt.plot(resampled_t, resampled_signal, color='b', label='resampled signal')
plt.plot(resampled_t, low_pass_filtered_signal, color='r', label='filtered signal')
plt.xlabel('Time in seconds')
plt.ylabel('Amplitude')
plt.title('Time domain representation of Resampled and Filtered Signal')
plt.legend()

In [ ]:
signal_length = len(resampled_signal)  
L_DFT = 2**int(np.ceil(np.log2(signal_length)))  

# perform DFT using the FFT algorithm
spectrum = np.fft.fft(resampled_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/target_rate) # the order of frequency is correspond to the output of FFT
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

low_pass_signal_length = len(low_pass_filtered_signal)
low_pass_L_DFT = 2**int(np.ceil(np.log2(low_pass_signal_length)))  
spectrum_filtered = np.fft.fft(low_pass_filtered_signal, low_pass_L_DFT)
magnitude_spectrum_filtered = np.abs(spectrum_filtered)
frequency_filtered = np.fft.fftfreq(low_pass_L_DFT, d=1/target_rate) # the order of frequency is correspond to the output of FFT
positive_frequency_filtered = frequency_filtered[:low_pass_L_DFT//2]
positive_magnitude_spectrum_filtered = magnitude_spectrum_filtered[:low_pass_L_DFT//2]

# plot the positive frequency domain of the speech signal
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.ylim(0, 500)
plt.title('Frequency domain representation of the resampled signal')

# plot the positive frequency domain of the filtered speech signal
plt.subplot(1, 2, 2)
plt.plot(positive_frequency_filtered, positive_magnitude_spectrum_filtered)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.ylim(0, 500)
plt.title('Frequency domain representation of the low pass filtered resampled signal')

In [ ]:
# plot the original and high pass filtered speech signal time-frequency diagram
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.specgram(resampled_signal, Fs=target_rate, NFFT=512, noverlap=256, cmap='viridis', scale='dB', vmin=-240, vmax=-40)
plt.title('Original spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')

plt.subplot(1, 2, 2)
plt.specgram(low_pass_filtered_signal, Fs=target_rate, NFFT=512, noverlap=256, cmap='viridis', scale='dB', vmin=-240, vmax=-40)
plt.title('Low pass filtered spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ') 

In [ ]:
ipd.Audio(resampled_signal, rate=target_rate)

In [ ]:
ipd.Audio(low_pass_filtered_signal, rate=target_rate)

### Band-stop filter

In [ ]:
filter_sampling_rate = 8000
nyquist = 0.5 * filter_sampling_rate
cutoff_freq = [300, 1100]
W_act = [fc / nyquist for fc in cutoff_freq]
# we need two frequencies for the tolerance scheme, we chose one of them a bit smaller than W_act and the other a bit larger than o.5, we define 
# a bit is equal to 0.025
W_bit = 0.025
Wp_1 = W_act[0] - W_bit
Ws_1 = W_act[0] + W_bit  
Wp_2 = W_act[1] + W_bit
Ws_2 = W_act[1] - W_bit
Wp = [Wp_1, Wp_2]
Ws = [Ws_1, Ws_2]
Rp_lin = 0.9  # allowed ripples in the pass band area
Rs_lin = 0.1  # allowed ripples in the stop band area

# draw the tolerance scheme
def plot_tolerance_scheme_band_stop(Wp, Ws, Rp_lin, Rs_lin):
    dh1x=[0,Ws[0]];  dh1y=[1,1];            
    dh2x=[0,Wp[0]];  dh2y=[Rp_lin,Rp_lin];
    dh3x=[Ws[1],1];  dh3y=[1,1];
    dh4x=[Wp[1],1];  dh4y=[Rp_lin,Rp_lin];
    dh5x=[Ws[0],Ws[1]];  dh5y=[Rs_lin, Rs_lin];
    dh6x=[Wp[0],Wp[1]];  dh6y=[0,0];
    sh1x=[Wp[0],Wp[0]];  sh1y=[Rp_lin,0]; 
    sh2x=[Ws[0],Ws[0]];  sh2y=[1,Rs_lin]; 
    sh3x=[Ws[1],Ws[1]]; sh3y=[1,Rs_lin];
    sh4x=[Wp[1],Wp[1]]; sh4y=[Rp_lin,0];
    # plot the actual lines
    plt.plot(dh1x,dh1y,'k--',dh2x,dh2y,'k--',dh3x,dh3y,'k--',dh4x,dh4y,'k--',dh5x,dh5y,'k--',
             dh6x,dh6y,'k--',sh1x,sh1y,'k--',sh2x,sh2y,'k--',sh3x,sh3y,'k--',sh4x,sh4y,'k--')
    plt.xlabel('Frequency $\Omega/\pi$')
plot_tolerance_scheme_band_stop(Wp, Ws, Rp_lin, Rs_lin)

In [ ]:
Rp = -20 * np.log10(Rp_lin)
Rs = -20 * np.log10(Rs_lin)
N, Wn = signal.buttord(Wp, Ws, Rp, Rs)
print('The minimum possible filter order to fulfil the tolerance scheme is '+str(N)+'.')
print('The 1st cut-off frequency which will be {:.2f}.'.format(Wn[0])) 
print('The 2nd cut-off frequency which will be {:.2f}.'.format(Wn[1]))

b, a = signal.butter(N, Wn, 'bandstop')
f,h=signal.freqz(b,a)
omega=np.linspace(0,1,len(f))

# plot the frequency response
plot_tolerance_scheme_band_stop(Wp, Ws, Rp_lin, Rs_lin)
plt.plot([Wn[0],Wn[0]],[0,1],color='red',ls=':',label='cutoff frequency 1')
plt.plot([Wn[1],Wn[1]],[0,1],color='green',ls=':',label='cutoff frequency 2')
plt.plot(omega, np.abs(h), lw=2, label='Butterworth bandstop filter')
plt.title('Butterworth bandstop filter of order ' + str(N))
plt.ylabel('Amplitude $|h(e^{j \Omega})|$')
plt.legend()

In [ ]:
# plot zeros and poles in the z plane
zplane(np.roots(b), np.roots(a), filter_type='band stop')

In [ ]:
band_stop_filtered_signal = signal.filtfilt(b, a, resampled_signal)
plt.figure(figsize=(12, 6))
plt.plot(resampled_t, resampled_signal, color='b', label='resampled signal')
plt.plot(resampled_t, band_stop_filtered_signal, color='r', label='filtered signal')
plt.xlabel('Time in seconds')
plt.ylabel('Amplitude')
plt.title('Time domain representation of Resampled and Filtered Signal')
plt.legend()

In [ ]:
signal_length = len(resampled_signal)  
L_DFT = 2**int(np.ceil(np.log2(signal_length)))  

# perform DFT using the FFT algorithm
spectrum = np.fft.fft(resampled_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/target_rate) # the order of frequency is correspond to the output of FFT
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

band_stop_signal_length = len(band_stop_filtered_signal)
band_stop_L_DFT = 2**int(np.ceil(np.log2(band_stop_signal_length)))  
spectrum_filtered = np.fft.fft(band_stop_filtered_signal, band_stop_L_DFT)
magnitude_spectrum_filtered = np.abs(spectrum_filtered)
frequency_filtered = np.fft.fftfreq(band_stop_L_DFT, d=1/target_rate) # the order of frequency is correspond to the output of FFT
positive_frequency_filtered = frequency_filtered[:band_stop_L_DFT//2]
positive_magnitude_spectrum_filtered = magnitude_spectrum_filtered[:band_stop_L_DFT//2]

# plot the positive frequency domain of the speech signal
plt.figure(figsize=(17, 6))
plt.subplot(1, 2, 1)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.ylim(0, 500)
plt.title('Frequency domain representation of the resampled signal')

# plot the positive frequency domain of the filtered speech signal
plt.subplot(1, 2, 2)
plt.plot(positive_frequency_filtered, positive_magnitude_spectrum_filtered)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.ylim(0, 500)
plt.title('Frequency domain representation of the band stop filtered resampled signal')

In [ ]:
# plot the original and band stop filtered speech signal time-frequency diagram
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.specgram(resampled_signal, Fs=target_rate, NFFT=512, noverlap=256, cmap='viridis', scale='dB', vmin=-240, vmax=-40)
plt.title('Original spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')

plt.subplot(1, 2, 2)
plt.specgram(band_stop_filtered_signal, Fs=target_rate, NFFT=512, noverlap=256, cmap='viridis', scale='dB', vmin=-240, vmax=-40)
plt.title('Band stop filtered spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ') 

In [ ]:
ipd.Audio(resampled_signal, rate=target_rate)

In [ ]:
ipd.Audio(band_stop_filtered_signal, rate=target_rate)

### Band pass filter

In [ ]:
filter_sampling_rate = 8000
nyquist = 0.5 * filter_sampling_rate
cutoff_freq = [300, 3400]
W_act = [fc / nyquist for fc in cutoff_freq]
# we need two frequencies for the tolerance scheme, we chose one of them a bit smaller than W_act and the other a bit larger than o.5, we define 
# a bit is equal to 0.025
W_bit = 0.025
Wp_1 = W_act[0] + W_bit 
Ws_1 = W_act[0] - W_bit  
Wp_2 = W_act[1] - W_bit
Ws_2 = W_act[1] + W_bit
Wp = [Wp_1, Wp_2]
Ws = [Ws_1, Ws_2]
Rp_lin = 0.9  # allowed ripples in the pass band area
Rs_lin = 0.1  # allowed ripples in the stop band area

# draw the tolerance scheme
def plot_tolerance_scheme_band_pass(Wp, Ws, Rp_lin, Rs_lin):
    dh1x=[0,Ws[0]];  dh1y=[Rs_lin,Rs_lin];            
    dh2x=[0,Wp[0]];  dh2y=[0,0];
    dh3x=[Ws[1],1];  dh3y=[Rs_lin,Rs_lin];
    dh4x=[Wp[1],1];  dh4y=[0,0];
    dh5x=[Ws[0],Ws[1]];  dh5y=[1,1];
    dh6x=[Wp[0],Wp[1]];  dh6y=[Rp_lin,Rp_lin];
    sh1x=[Wp[0],Wp[0]];  sh1y=[Rp_lin,0]; 
    sh2x=[Ws[0],Ws[0]];  sh2y=[1,Rs_lin]; 
    sh3x=[Ws[1],Ws[1]]; sh3y=[1,Rs_lin];
    sh4x=[Wp[1],Wp[1]]; sh4y=[Rp_lin,0];
    # plot the actual lines
    plt.plot(dh1x,dh1y,'k--',dh2x,dh2y,'k--',dh3x,dh3y,'k--',dh4x,dh4y,'k--',dh5x,dh5y,'k--',
             dh6x,dh6y,'k--',sh1x,sh1y,'k--',sh2x,sh2y,'k--',sh3x,sh3y,'k--',sh4x,sh4y,'k--')
    plt.xlabel('Frequency $\Omega/\pi$')
plot_tolerance_scheme_band_pass(Wp, Ws, Rp_lin, Rs_lin)

In [ ]:
Rp = -20 * np.log10(Rp_lin)
Rs = -20 * np.log10(Rs_lin)
N, Wn = signal.buttord(Wp, Ws, Rp, Rs)
print('The minimum possible filter order to fulfil the tolerance scheme is '+str(N)+'.')
print('The 1st cut-off frequency which will be {:.2f}.'.format(Wn[0])) 
print('The 2nd cut-off frequency which will be {:.2f}.'.format(Wn[1]))

b, a = signal.butter(N, Wn, 'band')
f,h=signal.freqz(b,a)
omega=np.linspace(0,1,len(f))

# plot the frequency response
plot_tolerance_scheme_band_pass(Wp, Ws, Rp_lin, Rs_lin)
plt.plot([Wn[0],Wn[0]],[0,1],color='red',ls=':',label='cutoff frequency 1')
plt.plot([Wn[1],Wn[1]],[0,1],color='green',ls=':',label='cutoff frequency 2')
plt.plot(omega, np.abs(h), lw=2, label='Butterworth bandpass filter')
plt.title('Butterworth bandpass filter of order ' + str(N))
plt.ylabel('Amplitude $|h(e^{j \Omega})|$')
plt.legend()

In [ ]:
# plot zeros and poles in the z plane
zplane(np.roots(b), np.roots(a), filter_type='band pass')

In [ ]:
band_pass_filtered_signal = signal.filtfilt(b, a, resampled_signal)
plt.figure(figsize=(12, 6))
plt.plot(resampled_t, resampled_signal, color='b', label='resampled signal')
plt.plot(resampled_t, band_pass_filtered_signal, color='r', label='filtered signal')
plt.xlabel('Time in seconds')
plt.ylabel('Amplitude')
plt.title('Time domain representation of Resampled and Filtered Signal')
plt.legend()

In [ ]:
signal_length = len(resampled_signal)  
L_DFT = 2**int(np.ceil(np.log2(signal_length)))  

# perform DFT using the FFT algorithm
spectrum = np.fft.fft(resampled_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/target_rate) # the order of frequency is correspond to the output of FFT
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

band_pass_signal_length = len(band_pass_filtered_signal)
band_pass_L_DFT = 2**int(np.ceil(np.log2(band_pass_signal_length)))  
spectrum_filtered = np.fft.fft(band_pass_filtered_signal, band_pass_L_DFT)
magnitude_spectrum_filtered = np.abs(spectrum_filtered)
frequency_filtered = np.fft.fftfreq(band_pass_L_DFT, d=1/target_rate) # the order of frequency is correspond to the output of FFT
positive_frequency_filtered = frequency_filtered[:band_pass_L_DFT//2]
positive_magnitude_spectrum_filtered = magnitude_spectrum_filtered[:band_pass_L_DFT//2]

# plot the positive frequency domain of the speech signal
plt.figure(figsize=(17, 6))
plt.subplot(1, 2, 1)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.ylim(0, 500)
plt.title('Frequency domain representation of the resampled signal')

# plot the positive frequency domain of the filtered speech signal
plt.subplot(1, 2, 2)
plt.plot(positive_frequency_filtered, positive_magnitude_spectrum_filtered)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.ylim(0, 500)
plt.title('Frequency domain representation of the band pass filtered resampled signal')

In [ ]:
# plot the original and band pass filtered speech signal time-frequency diagram
plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.specgram(resampled_signal, Fs=target_rate, NFFT=512, noverlap=256, cmap='viridis', scale='dB', vmin=-240, vmax=-40)
plt.title('Original spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')

plt.subplot(1, 2, 2)
plt.specgram(band_pass_filtered_signal, Fs=target_rate, NFFT=512, noverlap=256, cmap='viridis', scale='dB', vmin=-240, vmax=-40)
plt.title('Band pass filtered spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ') 

In [ ]:
ipd.Audio(resampled_signal, rate=target_rate)

In [ ]:
ipd.Audio(band_pass_filtered_signal, rate=target_rate)

<br>
<div style="border: 2px solid #999; padding: 10px; background: #9696ff;">
    
**Question Q2:** 

* Explain the behaviour of the designed band-stop filter, i.e. describe (briefly) what you can see in the generated plots. If you didn't generate plots you can explain what you would expect to see.

    
</div>

**Answer to Question Q2:**

<span style="font-weight:bold;color:orange">
A band-stop filter attenuates frequencies between two cut-off frequencies (defining the stop-band) while allowing frequencies outside this range (in the lower and higher pass-bands) to pass through with little attenuation. <br><br>
According to the Poles-Zeros graph, we observed that there are two zeros placed on the unit circle, representing the stop-band where frequencies are attenuated. The poles are located inside the unit circle, ensure stability and maintain the pass-band behaviour, allowing signals outside the stop-band to pass through with little attenuation.<br><br>
According to the frequency-domain graph, the frequencies withinn the stop-band (300-1100 HZ) have been significantly attenuated, the frequencies below 300HZ or above 1100 HZ remain largely affected. This behavior is consistent with the expected behavior of a band-stop filter.<br><br>
According to the time-frequency graph, frequencies within the stop-band (300-1100 HZ) have been significantly attenuated, as shown by the dark blue regions indicating lower levels in dB. Frequencies outside the stop-bandremain relatively unaffected, as shown by brighter region.
</span>

<br>
<div style="border: 2px solid #999; padding: 10px; background: #9696ff;">
    
**Question Q3:**
    
* Which sounds are most affected when the low-pass cut-off frequency is set to around $500$
Hz - vowels or consonants - and why?
    
</div>

**Answer to Question Q3:**

<span style="font-weight:bold;color:orange">
Consonants are most affected when the low pass cut-off frequency is set to around 500 HZ. Vowels are primarily composed of low-frequency components. The fundamental frequency of vowels and the first formant for most vowels like /i/, /u/, /ʊ/ and /ɛ/ lie below 500 HZ. These low frequencies are relatively unaffected by a 500 HZ low-pass filter, allowing vowels to remain recognizable even if higher formants (which may exceed 500 Hz) are attenuated. However, consonants contain a significant amount of high-frequency components, usually distributed between 1000 HZ to 8000 HZ, which are largly elimiated by 500 low pass filter, making consonants hard to hear or recognize. Therefore, consonants are most affected by a 500 HZ low pass filter.
</span>

# Audio Effects

## Low-Frequency Oscillator

Many ‘voice effects (FXs)’ are achieved by modifying some characteristic of the speech using a low-frequency oscillator or *LFO*. LFOs typically have two controls: speed (which is specified by the frequency in Hertz) and depth (which specifies the magnitude of the effect). The following tasks will require several LFOs, so it makes sense to implement one in the following.

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T4:**
    
* Implement a Low Frequency Oscillator as a function `lfo()` as described below. Visualise that your function works by generating a sine and a square wave of frequency $5$ Hz and length $2$ seconds with different depths.
    
Note: There will be an extra point in the marking if you **do not** use the `scipy` library to solve this task.
    
</div>

In [ ]:
def lfo_no_scipy(speed_hz, depth, num_samples, fs=44100, square_curve=False):
    '''
    Low-frequency oscillator
    
    Parameters
    ----------
    speed_hz : float
       frequency of generated signal in Hertz
    depth : float
        magnitude of the effect
    num_samples : int
        length of the signal in samples
    fs : float, optional 
        sampling frequency in Hz, default 44100
    square_curve : boolean, optional (default: False)
        generate square wave if true, generate sine wave if false

    Example use:
    -------
        sig_square = lfo(speed_hz=5, depth=0.7, num_samples=88200, fs=44100, square_curve=True)
    '''
    # Generate time vector
    t = np.linspace(0, int(num_samples/fs), num_samples, endpoint=False)  
    if square_curve:
        lfo_signal = depth * np.sign(np.sin(2 * np.pi * speed_hz * t))
    else:
        lfo_signal = depth * np.sin(2 * np.pi * speed_hz * t)
        
    return lfo_signal, t

In [ ]:
# Use lfo_no_scipy function to visualize LFO
fs = 44100  
duration = 2
num_samples = int(fs * duration)
speed_hz = 5  # LFO frequency (5 Hz)

plt.figure(figsize=(15, 10))
# Generate sine signal with amplitude 0.5 (depth=0.5)
plt.subplot(3, 2, 1)
sine_wave_half, sine_wave_half_t = lfo_no_scipy(speed_hz=speed_hz, depth=0.5, num_samples=num_samples, fs=fs, square_curve=False)
plt.plot(sine_wave_half_t, sine_wave_half)
plt.title('Sine wave with amplitude 0.5')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid()

# Generate sine signal with amplitude 1 (depth=1)
plt.subplot(3, 2, 3)
sine_wave_1, sine_wave_1_t = lfo_no_scipy(speed_hz=speed_hz, depth=1, num_samples=num_samples, fs=fs, square_curve=False)
plt.plot(sine_wave_1_t, sine_wave_1)
plt.title('Sine wave with amplitude 1')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid()

# Generate sine signal with amplitude 2 (depth=2)
plt.subplot(3, 2, 5)
sine_wave_2, sine_wave_2_t = lfo_no_scipy(speed_hz=speed_hz, depth=2, num_samples=num_samples, fs=fs, square_curve=False)
plt.plot(sine_wave_2_t, sine_wave_2)
plt.title('Sine wave with amplitude 2')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid()

# Generate square signal with amplitude 0.5 (depth=0.5)
plt.subplot(3, 2, 2)
square_wave_half, square_wave_half_t = lfo_no_scipy(speed_hz=speed_hz, depth=0.5, num_samples=num_samples, fs=fs, square_curve=True)
plt.plot(square_wave_half_t, square_wave_half)
plt.title('Square wave with amplitude 0.5')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid()

# Generate square signal with amplitude 1 (depth=1)
plt.subplot(3, 2, 4)
square_wave_1, square_wave_1_t = lfo_no_scipy(speed_hz=speed_hz, depth=1, num_samples=num_samples, fs=fs, square_curve=True)
plt.plot(square_wave_1_t, square_wave_1)
plt.title('Square wave with amplitude 1')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid()

# Generate square signal with amplitude 2 (depth=2)
plt.subplot(3, 2, 6)
square_wave_2, square_wave_2_t = lfo_no_scipy(speed_hz=speed_hz, depth=2, num_samples=num_samples, fs=fs, square_curve=True)
plt.plot(square_wave_2_t, square_wave_2)
plt.title('Square wave with amplitude 2')
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid()

plt.tight_layout()
plt.show()

Although your function outputs audio, you are unlikely to be able to hear it as the frequency is so low. However, you can check that it is functioning correctly by combining it with other audio signals as we will do in the following.

## Amplitude Modulation - Tremolo

*Tremolo* is one of the most basic voice manipulations that makes use of an LFO. In this effect, the amplitude of a speech signal is [modulated](https://en.wikipedia.org/wiki/Amplitude_modulation), i.e. the speech waveform is multiplied by a variable gain that ranges between $1-$ `modulation_depth` and $1$. This means that if the `modulation_depth` equals $1$, the variable gain varies between $1-1=0$ and $1$. 

Your LFO outputs an audio signal between `-depth` and `+depth` which is different from the `modulation_depth` above. So, in order to modulate the amplitude of the speech correctly, the output of the LFO has to be scaled and applied appropriately.

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T5:**
    
* Implement  a function `tremolo()` using your function `lfo()` and modulate the amplitude of the speech signal. 
* Experiment with different settings for `speed` and `modulation_depth`. In particular, note that a square wave with a *speed* between $3$ and $4$ Hz (and `modulation_depth` = $1$) has a very destructive effect on the intelligibility of the output. This is because $3-4$ Hz corresponds to the typical syllabic rate of speech.
* Proof that the effect works by a proper visualisation of the filtered speech signal and describe what can be observed and perceived.
    
</div>

In [ ]:
def tremolo(signal, fs, speed_hz, modulation_depth, square_curve=False):
    '''
    Applies a tremolo effect to a signal
    
    Parameters
    ----------
    signal : float
       input signal to which the effect should be applied
    fs : int
        sampling frequency in Hz
    speed_hz : float
       frequency of LFO and by this also the effect
    modulation_depth : float
        magnitude of the effect
    square_curve : boolean, optional
        generate square wave if true, generate sine wave if false
    
    Return
    ----------
    signal after application of tremolo effect
    
    Example use:
    -------
        signal_tremolo = tremolo(signal=audio_in, fs=fs, speed_hz=10, modulation_depth=1, square_curve=False)
    '''
    
    number_samples = len(signal)
    lfo_output = lfo_no_scipy(speed_hz=speed_hz, depth=modulation_depth, num_samples=number_samples, fs=fs, square_curve=square_curve)[0]
    lfo_output_scaled = (lfo_output + modulation_depth)/(2 * modulation_depth)
    variable_gain = 1 - modulation_depth + modulation_depth * lfo_output_scaled
    tremolo_signal = signal * variable_gain
    return tremolo_signal

In [ ]:
# plot the original speech signal time domain diagram
speech_signal_time = len(speech_signal)/speech_signal_fs
t = np.linspace(0, speech_signal_time, len(speech_signal), endpoint=False)
plt.figure(figsize=(10,4))
plt.plot(t, speech_signal)
plt.xlabel('Time in second')
plt.ylabel('Amplitude')
plt.title('Time domain representation of the signal')
plt.grid(True)

In [ ]:
# Changing speed_hz to show the effect of Tremolo signal
plt.figure(figsize=(18, 5*3))
speed_hz_list = [1, 5, 10]
tremelo_speed_sign_list = []
for i, speed_hz in enumerate(speed_hz_list):
    plt.subplot(3, 3, i*3+1)
    tremolo_signal = tremolo(signal=speech_signal, fs=speech_signal_fs, speed_hz=speed_hz, modulation_depth=1, square_curve=True)
    tremelo_speed_sign_list.append(tremolo_signal)
    plt.plot(t, tremolo_signal)
    plt.title(f"Tremolo signal")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True)

    signal_length = len(tremolo_signal)  
    L_DFT = 2**int(np.ceil(np.log2(signal_length)))  
    # perform DFT using the FFT algorithm
    spectrum = np.fft.fft(tremolo_signal, L_DFT)
    magnitude_spectrum = np.abs(spectrum)
    frequency = np.fft.fftfreq(L_DFT, d=1/speech_signal_fs) 
    positive_frequency = frequency[:L_DFT//2]
    positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

    # plot the positive frequency domain of the speech signal
    plt.subplot(3, 3, i*3+2)
    plt.plot(positive_frequency, positive_magnitude_spectrum)
    plt.xlabel('Frequency in HZ')
    plt.ylabel('Amplitude')
    plt.title('Frequency domain representation of the tremolo signal')

    # plot the tremolo signal time-frequency diagram
    plt.subplot(3, 3, i*3+3)
    # avoid divided by 0, we add a small number 
    plt.specgram(tremolo_signal+1e-10, Fs=speech_signal_fs, NFFT=512, noverlap=256, cmap='viridis', scale='dB')
    plt.title('Original spectrogram with DFT length of 512')
    plt.colorbar(label='dB')
    plt.xlabel('Time in seconds')
    plt.ylabel('Frequency in HZ')

In [ ]:
# Changing depth to show the effect of Tremolo signal
plt.figure(figsize=(18, 5*3))
depth_list = [1, 5, 10]
tremelo_depth_sign_list = []
for i, depth in enumerate(speed_hz_list):
    plt.subplot(3, 3, i*3+1)
    tremolo_signal = tremolo(signal=speech_signal, fs=speech_signal_fs, speed_hz=5, modulation_depth=depth, square_curve=True)
    tremelo_speed_sign_list.append(tremolo_signal)
    plt.plot(t, tremolo_signal)
    plt.title(f"Tremolo signal")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True)

    signal_length = len(tremolo_signal)  
    L_DFT = 2**int(np.ceil(np.log2(signal_length)))  
    # perform DFT using the FFT algorithm
    spectrum = np.fft.fft(tremolo_signal, L_DFT)
    magnitude_spectrum = np.abs(spectrum)
    frequency = np.fft.fftfreq(L_DFT, d=1/speech_signal_fs) 
    positive_frequency = frequency[:L_DFT//2]
    positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

    # plot the positive frequency domain of the speech signal
    plt.subplot(3, 3, i*3+2)
    plt.plot(positive_frequency, positive_magnitude_spectrum)
    plt.xlabel('Frequency in HZ')
    plt.ylabel('Amplitude')
    plt.title('Frequency domain representation of the tremolo signal')

    # plot the tremolo signal time-frequency diagram
    plt.subplot(3, 3, i*3+3)
    # avoid divided by 0, we add a small number 
    plt.specgram(tremolo_signal+1e-10, Fs=speech_signal_fs, NFFT=512, noverlap=256, cmap='viridis', scale='dB')
    plt.title('Original spectrogram with DFT length of 512')
    plt.colorbar(label='dB')
    plt.xlabel('Time in seconds')
    plt.ylabel('Frequency in HZ')

In [ ]:
# show the destructive effect of square wave with a speed between 3 and 4 Hz, depth=1
plt.figure(figsize=(18, 5))
plt.subplot(1, 3, 1)
tremolo_signal = tremolo(signal=speech_signal, fs=speech_signal_fs, speed_hz=3.5, modulation_depth=1, square_curve=True)
plt.plot(t, tremolo_signal)
plt.title(f"Tremolo signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.grid(True)

signal_length = len(tremolo_signal)  
L_DFT = 2**int(np.ceil(np.log2(signal_length)))  
# perform DFT using the FFT algorithm
spectrum = np.fft.fft(tremolo_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/speech_signal_fs) 
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

# plot the positive frequency domain of the speech signal
plt.subplot(1, 3, 2)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Frequency domain representation of the tremolo signal')

# plot the tremolo signal time-frequency diagram
plt.subplot(1, 3, 3)
# avoid divided by 0, we add a small number 
plt.specgram(tremolo_signal+1e-10, Fs=speech_signal_fs, NFFT=512, noverlap=256, cmap='viridis', scale='dB')
plt.title('Original spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')

In [ ]:
ipd.Audio(tremolo_signal, rate=speech_signal_fs)

<span style="font-weight:bold;color:orange">
The left plot shows the amplitude of the speech signal modulated by the tremolo effect over time. It represents the effect of a tremolo with speed_hz=3.5 and depth=1. We can observe periodic fluctuations in amplitude, which are characteristic of the tremolo effect. The tremolo signal has regions where the amplitude decreases significantly, which disrupts the continuity of the speech, leading to perceptual distortion.<br><br>

The middle plot shows the frequency domian representation of the speech signal modulated by the tremolo effect over time. Since the frequency domain is very wide, we cannot directly observe obvious effect. We can perceive additional low-frequency content.<br><br>

The right plot shows the spectrogram of the tremolo signal.The spectrogram reveals a periodic weakening and strengthening of frequencies (we can see alternating dark and bright vertical lines), corresponding to the amplitude modulation caused by the tremolo effect.
</span>

## Ring Modulation

Another basic effect is to multiply the speech signal by the output of an LFO. This is known as ‘ring modulation’.

Note: In the BBC TV series [Dr. Who](https://en.wikipedia.org/wiki/Doctor_Who), the voices of the alien [Daleks](https://en.wikipedia.org/wiki/Dalek) are generated by a ring modulator with an LFO set to around 30 Hz. The voice actors also spoke using a stilted monotonic intonation in order to enhance the effect. You can try this yourself by recording your own voice and applying the effect.

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T6 (Ring Modulation):**
    
* Implement a function `ring_modulation()` using your function `lfo()` and modulate the amplitude of the speech signal by multiplying with the LFO signal. 
* Experiment with different settings for `speed` and `depth`. Note how the timbre of the resulting sound is subtly different from *tremolo*.
* Proof that the effect works by a proper visualisation of the filtered speech signal and describe what can be observed and perceived.
    
</div>

In [ ]:
def ring_modulation(signal, fs, speed_hz, depth, square_curve=False):
    '''
    Applies a ring modulation effect to a signal
    
    Parameters
    ----------
    signal : float
       input signal to which the effect should be applied
    fs : int
        sampling frequency in Hz
    speed_hz : float
       frequency of LFO and by this also the effect
    depth : float
        magnitude of the effect
    square_curve : boolean, optional
        generate square wave if true, generate sine wave if false
    
    Return
    ----------
    signal after application of the ring modulation effect
    
    Example use:
    -------
        signal_ring_mod = ring_modulation(audio_in, fs, 10, 1, square_curve=False)
    '''
    num_samples = len(signal)
    lfo_output = lfo_no_scipy(speed_hz=speed_hz, depth=depth, num_samples=num_samples, fs=fs, square_curve=square_curve)[0]
    ring_modulation_signal = signal * lfo_output
    return ring_modulation_signal

In [ ]:
# Your code here to show an example of the Ring Modulation effect
# plot the original speech signal time domain diagram
speech_signal_time = len(speech_signal)/speech_signal_fs
t = np.linspace(0, speech_signal_time, len(speech_signal), endpoint=False)
plt.figure(figsize=(12,10))
plt.subplot(2, 1, 1)
plt.plot(t, speech_signal)
plt.xlabel('Time in second')
plt.ylabel('Amplitude')
plt.title('Time domain representation of the signal')
plt.grid(True)

In [ ]:
# Changing speed_hz to show the effect of Ring Modulation
plt.figure(figsize=(18, 5*3))
speed_hz_list = [1, 5, 10]
ring_speed_sign_list = []
for i, speed_hz in enumerate(speed_hz_list):
    plt.subplot(3, 3, i*3+1)
    ring_signal = ring_modulation(signal=speech_signal, fs=speech_signal_fs, speed_hz=speed_hz, depth=1, square_curve=False)
    ring_speed_sign_list.append(ring_signal)
    plt.plot(t, ring_signal)
    plt.title(f"Ring signal")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True)

    signal_length = len(ring_signal)  
    L_DFT = 2**int(np.ceil(np.log2(signal_length)))  
    # perform DFT using the FFT algorithm
    spectrum = np.fft.fft(ring_signal, L_DFT)
    magnitude_spectrum = np.abs(spectrum)
    frequency = np.fft.fftfreq(L_DFT, d=1/speech_signal_fs) 
    positive_frequency = frequency[:L_DFT//2]
    positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

    # plot the positive frequency domain of the speech signal
    plt.subplot(3, 3, i*3+2)
    plt.plot(positive_frequency, positive_magnitude_spectrum)
    plt.xlabel('Frequency in HZ')
    plt.ylabel('Amplitude')
    plt.title('Frequency domain representation of the ring signal')

    # plot the ring signal time-frequency diagram
    plt.subplot(3, 3, i*3+3)
    # avoid divided by 0, we add a small number 
    plt.specgram(ring_signal, Fs=speech_signal_fs, NFFT=512, noverlap=256, cmap='viridis', scale='dB')
    plt.title('Original spectrogram with DFT length of 512')
    plt.colorbar(label='dB')
    plt.xlabel('Time in seconds')
    plt.ylabel('Frequency in HZ')

In [ ]:
plt.figure(figsize=(18, 5*3))
depth_list = [1, 5, 10]
ring_depth_sign_list = []
for i, depth in enumerate(depth_list):
    plt.subplot(3, 3, i*3+1)
    ring_signal = ring_modulation(signal=speech_signal, fs=speech_signal_fs, speed_hz=5, depth=depth, square_curve=False)
    ring_depth_sign_list.append(ring_signal)
    plt.plot(t, ring_signal)
    plt.title(f"Ring signal")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.grid(True)

    signal_length = len(ring_signal)  
    L_DFT = 2**int(np.ceil(np.log2(signal_length)))  
    # perform DFT using the FFT algorithm
    spectrum = np.fft.fft(ring_signal, L_DFT)
    magnitude_spectrum = np.abs(spectrum)
    frequency = np.fft.fftfreq(L_DFT, d=1/speech_signal_fs) 
    positive_frequency = frequency[:L_DFT//2]
    positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

    # plot the positive frequency domain of the speech signal
    plt.subplot(3, 3, i*3+2)
    plt.plot(positive_frequency, positive_magnitude_spectrum)
    plt.xlabel('Frequency in HZ')
    plt.ylabel('Amplitude')
    plt.title('Frequency domain representation of the ring signal')

    # plot the ring signal time-frequency diagram
    plt.subplot(3, 3, i*3+3)
    # avoid divided by 0, we add a small number 
    plt.specgram(ring_signal, Fs=speech_signal_fs, NFFT=512, noverlap=256, cmap='viridis', scale='dB')
    plt.title('Original spectrogram with DFT length of 512')
    plt.colorbar(label='dB')
    plt.xlabel('Time in seconds')
    plt.ylabel('Frequency in HZ')

In [ ]:
ipd.Audio(ring_depth_sign_list[2], rate=speech_signal_fs)

## Frequency Shifting

Many Vocal FX are the result of altering the frequencies present, e.g. changing the pitch of a voice. There are many algorithms for frequency shifting. You have already implemented an approximate solution with your ring modulator.

For simplicity, the following function will be given implementing frequency shifting.

In [ ]:
# the following code in this cell is taken and slightly adapted from:
# https://gist.github.com/lebedov/4428122

import scipy.signal as sig
def nextpow2(n):
    '''Return the first integer N such that 2**N >= abs(n)'''
    return int(np.ceil(np.log2(np.abs(n))))

def frequency_shift(signal, fs, shift_amount):
    '''
    Shift the specified signal by the specified frequency.
    
    Parameters
    ----------
    signal : float
       input signal to which the effect should be applied
    fs : int
        sampling frequency in Hz
    shift_amount : float
       amount of frequency shift (in Hz)
   
    Return
    ----------
    signal after application of the frequency shifting effect
    
    Example use:
    -------
        signal_frequency_shifted = frequency_shift(audio_in, fs, 100)
    '''

    # Pad the signal with zeros to prevent the FFT invoked by the transform from
    # slowing down the computation:
    N_orig = len(signal)
    N_padded = 2 ** nextpow2(N_orig)
    t = np.arange(0, N_padded)
    
    # Hilbert transform to get the analytic signal
    analytic_signal = sig.hilbert(np.hstack((signal, np.zeros(N_padded - N_orig, signal.dtype))))  # np.hstack concat two signal
    frequency_shifted = analytic_signal * np.exp(2j * np.pi * shift_amount * t / fs)
    # Extract the real part and truncate to the original length
    shifted_frequency_signal = np.real(frequency_shifted[:N_orig])
    
    return shifted_frequency_signal

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T7 (Frequency Shifting):**
    
* Visualise the effect of the frequency shift effect using an appropriate spectral representation.
    
</div>

In [ ]:
# Visualise the signal in frequency domain
speech_signal_length = len(speech_signal)  # the length of the signal is 928086, we need to find the L_DFT that is the exponential of 2 and should be larger than the length of signal
L_DFT = 2**int(np.ceil(np.log2(speech_signal_length)))  #2**20

# perform DFT using the FFT algorithm
spectrum = np.fft.fft(speech_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/speech_signal_fs)
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]
plt.figure(figsize=(15, 15))

# plot the full frequency domain of the speech signal
plt.subplot(3, 2, 1)
plt.plot(frequency, magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Full frequency domain representation of the signal')

# plot the positive frequency domain of the speech signal
plt.subplot(3, 2, 2)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Positive frequency domain representation of the signal')

# Shift the wave
shifted_speech_signal = frequency_shift(signal=speech_signal, fs=speech_signal_fs, shift_amount=2000)
shifted_speech_signal_length = len(shifted_speech_signal) # the length of the shifted signal is 928086
# we need to find the L_DFT that is the exponential of 2 and should be larger than the length of signal

L_DFT_shifted = 2**int(np.ceil(np.log2(shifted_speech_signal_length)))  #2**20
# perform DFT using the FFT algorithm
spectrum_shifted = np.fft.fft(shifted_speech_signal, L_DFT_shifted)
magnitude_spectrum_shifted = np.abs(spectrum_shifted)
frequency_shifted = np.fft.fftfreq(L_DFT_shifted, d=1/speech_signal_fs)
positive_frequency_shifted = frequency_shifted[:L_DFT_shifted//2]
positive_magnitude_spectrum_shifted = magnitude_spectrum_shifted[:L_DFT_shifted//2]

# plot the full frequency domain of the shifted speech signal
plt.subplot(3, 2, 3)
plt.plot(frequency_shifted, magnitude_spectrum_shifted)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Full frequency domain representation of the shifted signal')

# plot the positive frequency domain of the shifted speech signal
plt.subplot(3, 2, 4)
plt.plot(positive_frequency_shifted, positive_magnitude_spectrum_shifted)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Positive frequency domain representation of the shifted signal')

plt.subplot(3, 2, 5)
plt.plot(frequency, magnitude_spectrum, color='r', label='original speech signal')
plt.plot(frequency_shifted, magnitude_spectrum_shifted, color='g', label='shifted speech signal')
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Full frequency domain comparation')
plt.legend()

plt.subplot(3, 2, 6)
plt.plot(positive_frequency, positive_magnitude_spectrum, color='r', label='original speech signal')
plt.plot(positive_frequency_shifted, positive_magnitude_spectrum_shifted, color='g', label='shifted speech signal')
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Positive frequency domain comparation')
plt.legend()

In [ ]:
ipd.Audio(shifted_speech_signal, rate=speech_signal_fs)

<br>
<div style="border: 2px solid #999; padding: 10px; background: #9696ff;">
    
**Question Q4:**

* COM3502-4502-6502: Why can the voice be shifted up in frequency much further than
it can be shifted down in frequency before it becomes severely distorted? Hint: Calculate a spectrum plot if the answer is not immediately clear to you.
* COM4502-6502 ONLY: Your frequency shifter changes all the frequencies present in an input signal. How might it be possible to change the pitch of a voice without altering the formant frequencies?
    
</div>

**Answer to Question Q4:**

<span style="font-weight:bold;color:orange">
    We know that the primary frequency range of human speech is about 300 HZ to 3400 HZ, which is the main component of speech. 300 HZ is the lowest effective frequency of speech, and frequencies below this range contribute little to the intelligibility. The audible range of the human ear is 20 HZ to 20000HZ.<br><br>

When we shift up the frequency, for example 1000HZ, the new range can be 1300HZ and 4400HZ, this range is still in the audible range of the human ear. At this time, the spectrum structure of the speech signal remains intact, although the overall frequency becomes higher, but the relative spectrum distribution is unchanged, so human can still understand the signal.<br><br>

When we shift down the frequency, frequency components will close to 0 HZ or even negative frequencies. However, in the actual signal, there is no negative frequencies, negative frequencies will be folded onto the positive frequency axis, causing aliasing. Downward shift will also cause the original high frequency part to enter the low frequency region, and even lose to the range of human ears (below 20 Hz).<br><br>

Therefore, voiced can be shifted up in frequency much further than it can be shifted down in frequency.
</span>

## Harmony Effect

A classic ‘robotic’ voice can be achieved by simply adding frequency-shifted speech back to the unprocessed original. This effect is known as ‘harmony’. However, rather than simply adding the signals in equal amounts, we will implement a more general-purpose approach.

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T8:**
    
* Implement a function `mixer()` that adds the original speech with the manipulated speech in different proportions. 
* Implement a function `harmony()` that mixes the input signal with a frequency-shifted version of itself (using the functions `mixer()` and `frequency_shift()`). With your mixer at the $50$-$50$ setting, experiment with different frequency shifts in order to produce the best robotic-sounding output. Report "your optimal" setting. 
</div>

In [ ]:
def mixer(original_speech_signal, manipulated_speech_signal, percentage_l=0.5):
    mixed_speech_signal = percentage_l * original_speech_signal + (1-percentage_l) * manipulated_speech_signal
    return mixed_speech_signal

In [ ]:
def harmony(original_speech_signal, fs, shift_amount, percentage_l=0.5):
    shifted_speech_signal = frequency_shift(signal=original_speech_signal, fs=fs, shift_amount=shift_amount)
    harmony_speech_signal = mixer(original_speech_signal=original_speech_signal, manipulated_speech_signal=shifted_speech_signal, percentage_l=percentage_l)
    return harmony_speech_signal

In [ ]:
# Your code here to show an example of the harmony effect
shift_amount_list = np.arange(0, 800, 100)
harmony_speech_signal_list = []
for shift in shift_amount_list:
    harmony_speech_signal = harmony(original_speech_signal=speech_signal, fs=speech_signal_fs, shift_amount=shift, percentage_l=0.5)
    harmony_speech_signal_list.append(harmony_speech_signal)
print(harmony_speech_signal_list, len(harmony_speech_signal_list))

In [ ]:
ipd.Audio(speech_signal, rate=speech_signal_fs)

In [ ]:
ipd.Audio(harmony_speech_signal_list[0], rate=speech_signal_fs)

In [ ]:
ipd.Audio(harmony_speech_signal_list[1], rate=speech_signal_fs)

In [ ]:
ipd.Audio(harmony_speech_signal_list[2], rate=speech_signal_fs)

In [ ]:
ipd.Audio(harmony_speech_signal_list[3], rate=speech_signal_fs)

In [ ]:
ipd.Audio(harmony_speech_signal_list[4], rate=speech_signal_fs)

**Answer to question in Task T7:**

<span style="font-weight:bold;color:orange">
My optimal setting is: a frequency shift of 200 HZ with the mixer set to 50-50 proportion. <br>
Because 200HZ frequency shift introduces sufficient harmonic dissonance, producing a clear robotic tone without making the signal too distorted. I determined this optimal setting by systematically listening to the harmony speech signal with frequency shift from 0 to 800 with a footstep 100. <br><br>

At 100 HZ, some harmonic dissonance was introduced, but the human speech remained too intelligible.<br>
At 200 HZ, the robotic effect was perfect, achieving a balance between clarity and the desired robotic tone.<br>
Beyond 200 HZ, the speech signal became increasingly distorted, which were not performed better than 200 HZ.
</span>

## Frequency Modulation: Vibrato

Now that you have the ability to shift the frequencies in a speech signal, it is very easy to implement another common voice manipulation technique - *vibrato*. All that is required is for the frequency shifter to be controlled by the output of an LFO.



<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T9:**
    
* Implement a function `vibrato()` by connecting an LFO to your frequency shifter, and experiment with different values for speed and depth. Note that the LFO output will need to be scaled to provide an appropriate frequency shift range and then added to the output of the frequency shift.
    
</div>



In [ ]:
def frequency_shift_dynamic(signal, fs, shift_amount):
    # Pad the signal with zeros to prevent the FFT invoked by the transform from
    # slowing down the computation:
    N_orig = len(signal)
    N_padded = 2 ** nextpow2(N_orig)
    t = np.arange(0, N_padded)
    
    # Hilbert transform to get the analytic signal
    shift_amount = np.pad(shift_amount, (0, N_padded-len(shift_amount)), mode='constant')
    analytic_signal = sig.hilbert(np.hstack((signal, np.zeros(N_padded - N_orig, signal.dtype))))  # np.hstack concat two signal
    frequency_shifted = analytic_signal * np.exp(2j * np.pi * shift_amount * t / fs)
    # Extract the real part and truncate to the original length
    vibrato_signal = np.real(frequency_shifted[:N_orig])
    
    return vibrato_signal

In [ ]:
# Vibrato effect implementation
def vibrato(signal, speed_hz, depth, fs, square_curve=False):
    num_samples = len(signal)
    lfo_output = lfo_no_scipy(speed_hz=speed_hz, depth=depth, num_samples=num_samples, fs=fs, square_curve=square_curve)[0]
    vibrato_signal = frequency_shift_dynamic(signal=signal, fs=fs, shift_amount=lfo_output)

    return vibrato_signal

In [ ]:
# experiment with different values for speed and fixed depth
speed_list = np.arange(0, 5, 1)
vibrato_signal_speed_list = []
for speed in speed_list:
    vibrato_signal = vibrato(signal=speech_signal, speed_hz=speed, depth=0.5, fs=speech_signal_fs, square_curve=False)
    vibrato_signal_speed_list.append(vibrato_signal)

In [ ]:
# vibrato with depth=0.5, speed_hz=1
ipd.Audio(vibrato_signal_speed_list[0], rate=speech_signal_fs)

In [ ]:
# vibrato with depth=0.5, speed_hz=2
ipd.Audio(vibrato_signal_speed_list[1], rate=speech_signal_fs)

In [ ]:
# vibrato with depth=0.5, speed_hz=3
ipd.Audio(vibrato_signal_speed_list[2], rate=speech_signal_fs)

In [ ]:
# vibrato with depth=0.5, speed_hz=4
ipd.Audio(vibrato_signal_speed_list[3], rate=speech_signal_fs)

In [ ]:
# experiment with different values for depth and fixed speed
depth_list = np.arange(0, 1, 0.2)
vibrato_signal_depth_list = []
for depth in depth_list:
    vibrato_signal = vibrato(signal=speech_signal, speed_hz=3, depth=depth, fs=speech_signal_fs, square_curve=False)
    vibrato_signal_depth_list.append(vibrato_signal)

In [ ]:
# vibrato with speed_hz=3, depth=0.2
ipd.Audio(vibrato_signal_depth_list[1], rate=speech_signal_fs)

In [ ]:
# vibrato with speed_hz=3, depth=0.4
ipd.Audio(vibrato_signal_depth_list[2], rate=speech_signal_fs)

In [ ]:
# vibrato with speed_hz=3, depth=0.6
ipd.Audio(vibrato_signal_depth_list[3], rate=speech_signal_fs)

In [ ]:
# vibrato with speed_hz=3, depth=0.8
ipd.Audio(vibrato_signal_depth_list[4], rate=speech_signal_fs)

In [ ]:
# create a sine wave to implement vibrato effect
sine_wave_fs = 8000
sine_wave_time = 10
sine_wave_t = np.arange(0, sine_wave_time, 1 / sine_wave_fs)
sine_wave = np.sin(2 * np.pi * 50 * sine_wave_t)

In [ ]:
vibrato_sine = vibrato(signal=sine_wave, speed_hz=1, depth=0.1, fs=8000, square_curve=False)

In [ ]:
ipd.Audio(sine_wave, rate=sine_wave_fs)

In [ ]:
ipd.Audio(vibrato_sine, rate=sine_wave_fs)

## Time Delay Effect - Echo and Comb Filter

Many interesting voice FX can be achieved by delaying the signal and recombining it with itself. 

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T10:**
    
* Implement a function `echo()` which mixes a signal $s(t)$ with itself in a delayed version, i.e. $s(t-t_0)$. Experiment with various values for the delay $t_0$, and note the different effects you can achieve with delays 
  * below $20$ msecs
  * between $20$ and $100$ msecs, and 
  * above $100$ msecs.
</div>

In [ ]:
def echo(signal, fs, delay_ms, percentage=0.7):
    delay_samples = int(delay_ms * fs / 1000)  
    echoed_signal = np.zeros(len(signal) + delay_samples)  # Initialize echoed signal with zeros
    echoed_signal[:len(signal)] += signal  # Add original signal
    echoed_signal[delay_samples:] += percentage * signal
    total_samples = len(echoed_signal)
    duration = total_samples/fs
    t_echoed_signal = np.linspace(0, duration, len(echoed_signal), endpoint=False)
    return t_echoed_signal, echoed_signal

In [ ]:
# Your code here to show an example of the echo effect
speech_signal_time = len(speech_signal)/speech_signal_fs
t = np.linspace(0, speech_signal_time, len(speech_signal), endpoint=False)
plt.figure(figsize=(12,5))
plt.plot(t, speech_signal)
plt.xlabel('Time in second')
plt.ylabel('Amplitude')
plt.title('Time domain representation of the signal')
plt.grid(True)

delay_list = [10, 15, 50, 80, 150, 250]
num_delay = len(delay_list)
echoed_signal_list = []
plt.figure(figsize=(25, 6 * num_delay))
for i, delay in enumerate(delay_list):
    t_echoed_signal, echoed_signal = echo(signal=speech_signal, fs=speech_signal_fs, delay_ms=delay, percentage=0.7)
    echoed_signal_list.append(echoed_signal)
    plt.subplot(num_delay, 3, i*3 + 1)
    plt.plot(t_echoed_signal, echoed_signal)
    plt.xlabel('Time In Second')
    plt.ylabel('Amplitude')
    plt.title(f"Time domain (Delay={delay} ms)")
    plt.grid(True)

    # create the frequency domain representation of the signal
    echoed_signal_fs = speech_signal_fs
    echoed_signal_length = len(echoed_signal)
    L_DFT = 2**int(np.ceil(np.log2(echoed_signal_length)))
    freq_response = np.fft.fft(echoed_signal, L_DFT)
    magnitude = np.abs(freq_response)
    frequencies = np.fft.fftfreq(L_DFT, 1 / echoed_signal_fs)
    positive_frequencies = frequencies[:len(frequencies)//2]
    positive_magnitude = magnitude[:len(freq_response)//2]
    plt.subplot(num_delay, 3, i*3 + 2)
    plt.plot(positive_frequencies, positive_magnitude)
    plt.title(f"Frequency domain (Delay={delay} ms)")
    plt.xlabel("Frequency in HZ")
    plt.ylabel("Amplitude")
    plt.grid()

    plt.subplot(num_delay, 3, i*3 + 3)
    plt.specgram(echoed_signal, Fs=echoed_signal_fs, NFFT=1024, noverlap=512, cmap='viridis', scale='dB')
    plt.title(f"Spectrogram (Delay={delay} ms)")
    plt.colorbar(label='dB')
    plt.xlabel('Time in seconds')
    plt.ylabel('Frequency in HZ')


In [ ]:
# the original speech signal
ipd.Audio(speech_signal, rate=speech_signal_fs)

In [ ]:
# the echoed speech signal with delay 10ms
ipd.Audio(echoed_signal_list[0], rate=echoed_signal_fs)

In [ ]:
# the echoed speech signal with delay 15ms
ipd.Audio(echoed_signal_list[1], rate=echoed_signal_fs)

In [ ]:
# the echoed speech signal with delay 50ms
ipd.Audio(echoed_signal_list[2], rate=echoed_signal_fs)

In [ ]:
# the echoed speech signal with delay 80ms
ipd.Audio(echoed_signal_list[3], rate=echoed_signal_fs)

In [ ]:
# the echoed speech signal with delay 150ms
ipd.Audio(echoed_signal_list[4], rate=echoed_signal_fs)

In [ ]:
# the echoed speech signal with delay 250ms
ipd.Audio(echoed_signal_list[5], rate=echoed_signal_fs)

## Comb Filtering 

You should observe that with delays below $20$ msec in your function `echo()`, the signals combine to create a subtle ‘phasing’ effect. This is known as ‘comb filtering’ as the signal is effectively interfering with itself, and frequency components corresponding to multiples of the delay time are enhanced or cancelled out (due to ‘superposition’). Delays between $20$ and $100$ msecs give the effect of the voice being in a reverberant room. Delays above $100$ msecs sound like distant echoes.

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T11:**
    
* Visualise the impulse response of your function `echo()` as well as the transfer function. Can you give an explanation from having a look at the transfer function, why this effect would be called *comb filter*?
</div>

In [ ]:
# Create an impulse signal
length = 1000
impulse_position = 0
impulse_signal = np.zeros(length)
impulse_signal[impulse_position] = 1
impulse_signal_fs = 8000

# echo effect
plt.figure(figsize=(20, 6 * num_delay))
for i, delay in enumerate(delay_list):
    plt.subplot(num_delay, 2, i * 2+1)
    t_echoed_signal, impulse_response = echo(signal=impulse_signal, fs=impulse_signal_fs, delay_ms=delay, percentage=0.7)
    plt.plot(t_echoed_signal, impulse_response, label='Impulse Response')
    plt.title(f"Impulse Response of the Echo Function with delay {delay}ms")
    plt.xlabel("Time in seconds")
    plt.ylabel("Amplitude")
    plt.grid()
    
    # Impulse response of the Echo Function
    impulse_response_length = len(impulse_response)
    L_DFT = 2**int(np.ceil(np.log2(impulse_response_length)))
    freq_response = np.fft.fft(impulse_response, L_DFT)
    frequencies = np.fft.fftfreq(L_DFT, 1 / impulse_signal_fs)
    magnitude = np.abs(freq_response)
    positive_magnitude = magnitude[:len(freq_response)//2]
    positive_frequencies = frequencies[:len(freq_response)//2]
    plt.subplot(num_delay, 2, i * 2 + 2)
    plt.plot(positive_frequencies, positive_magnitude)
    plt.title(f"Magnitude Response of the Transfer Function with delay {delay}ms")
    plt.xlabel("Frequency in HZ")
    plt.ylabel("Amplitude")
    plt.grid()


**Answer to question in Task T11:**

<span style="font-weight:bold;color:orange">
The effect is called a comb filter because transfer function's magnitude response exhibits regularly spaced peaks and valleys across the frequency axis, the shape of these transfer function resembling the shape of comb.The peaks in the transfer function occur at frequencies where the original signal and delayed signal align in phase. This constructive interference amplifies specific frequency components. The valleys in the transfer function occur at frequencies where the original signal and the delayed signal are out of phase. This resulting in destructive interference and complete cancellation of those frequencies. <br><br>

The interference pattern (constructive and destructive) repeats periodically based on the delay time. As seen in transfer function graphs above, the evenly spaced peaks and valleys along the frequency domain create a comb-like shape.
</span>

## Flanger 

It is possible to use an LFO to vary the delay. The resulting effect is known as a *flanger*.

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T12:**
    
* Add an LFO to your ‘delay’ to create a ‘flanger’, and experiment with different settings. Note that you will need to scale the output of the LFO, and you will get different effects depending on whether the delayed signal is mixed with the original or not.
</div>

In [ ]:
def flanger(signal, max_delay_ms, fs, speed_hz, depth, mix_ratio, square_curve=True):
    num_samples = len(signal)
    lfo_output = lfo_no_scipy(speed_hz=speed_hz, depth=depth, num_samples=num_samples, fs=fs, square_curve=square_curve)[0]
    # negative delay is not meaningful in physics, therefore we scale the lfo output from (-depth, depth) to (0, 1)
    scaled_lfo_output = (lfo_output + depth)/(2 * depth)
    max_delay_samples = int(max_delay_ms / 1000 * fs)
    delay_samples = (scaled_lfo_output * max_delay_samples).astype(int)
    
    # create the flanger signal
    flanger_signal = np.zeros_like(signal)
    for n in range(num_samples):
        delay = delay_samples[n]
        if n - delay >= 0:
            flanger_signal[n] = signal[n] * (1 - mix_ratio) + signal[n - delay] * mix_ratio
        else:
            flanger_signal[n] = signal[n]  # Handle out-of-bounds delay
    return flanger_signal

In [ ]:
mix_ratio_list = np.arange(0, 1.2, 0.2)
flanger_signal_list = []
for mix_ratio in mix_ratio_list:
    flanger_signal = flanger(signal=speech_signal, max_delay_ms=15, fs=speech_signal_fs, speed_hz=10, depth=1, mix_ratio=mix_ratio, square_curve=False)
    flanger_signal_list.append(flanger_signal)

In [ ]:
for i, flanger_signal in enumerate(flanger_signal_list):
    plt.figure(figsize=(16, 6*len(flanger_signal_list)))
    plt.subplot(len(flanger_signal_list), 2, i*2+1)
    flanger_signal_length = len(flanger_signal)
    L_DFT = 2**int(np.ceil(np.log2(flanger_signal_length)))
    freq_response = np.fft.fft(flanger_signal, L_DFT)
    frequencies = np.fft.fftfreq(L_DFT, 1 / speech_signal_fs)
    magnitude = np.abs(freq_response)
    positive_magnitude = magnitude[:len(freq_response)//2]
    positive_frequencies = frequencies[:len(freq_response)//2]
    plt.plot(positive_frequencies, positive_magnitude)
    plt.title(f'Frequency representation of the Flanger signal with mix ratio={((i+1)*0.2):.2f}')
    plt.xlabel("Frequency in HZ")
    plt.ylabel("Amplitude")
    plt.grid()

    plt.subplot(len(flanger_signal_list), 2, i*2+2)
    plt.specgram(flanger_signal, Fs=speech_signal_fs, NFFT=1024, noverlap=512, cmap='viridis', scale='dB')
    plt.title(f"Spectrogram of the Flanger signal with mix ratio={((i+1)*0.2):.2f}")
    plt.colorbar(label='dB')
    plt.xlabel('Time in seconds')
    plt.ylabel('Frequency in HZ')

In [ ]:
# flanger signal with mix ratio=0.2
ipd.Audio(flanger_signal_list[0], rate=speech_signal_fs)

In [ ]:
# flanger signal with mix ratio=0.4
ipd.Audio(flanger_signal_list[1], rate=speech_signal_fs)

In [ ]:
# flanger signal with mix ratio=0.6
ipd.Audio(flanger_signal_list[2], rate=speech_signal_fs)

In [ ]:
# flanger signal with mix ratio=0.8
ipd.Audio(flanger_signal_list[3], rate=speech_signal_fs)

In [ ]:
# flanger signal with mix ratio=1.0
ipd.Audio(flanger_signal_list[4], rate=speech_signal_fs)

# Frequency Analysis

<br>
<div style="border: 2px solid #999; padding: 10px; background: #9696ff;">
    
**Question Q5:**

* COM3502-4502-6502: 
    * What does FFT stand for and what does an FFT do?
* COM4502-6502 ONLY: What is a DFT and how is it different from an FFT?
    
</div>

**Answer to Question Q5:**

<span style="font-weight:bold;color:orange">
FFT stands for Fast Fourier Transform. It is an efficient algorithm for computing the Discrete Fourier Transform (DFT) of a signal, which transforms a time-domain representation into its frequency-domain representation. To accelerate the computation, FFT leverages the structure of the DFT when the signal length is a power of 2. If the signal length is not a power of 2, zero-padding is applied to extend it to the nearest power of 2. This optimization reduces the computational complexity from  $O(N^2)$  for a regular DFT to  $O(NlogN)$ , making the transformation process significantly faster.
</span>

## Creating the Spectrogram Step-by-Step (Task T13 for COM4502-6502 only, Task T14 for all students)

The magnitude $|X[n, \ell]|$ of the STFT for all $n$ and $\ell$ is known as the [spectrogram](https://en.wikipedia.org/wiki/Spectrogram) of a signal. It is frequently used to analyse signals in the time-frequency domain, for instance by a [spectrum analyser](https://en.wikipedia.org/wiki/Spectrum_analyzer). It can be interpreted as a *image* of the signal with (block) time direction on the $x$ axis and (discrete) frequency $n$ on the y axis.

From Lab Sheet 3 we already know how to breack a long signal into block, a.k.a. frames.

<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task 13: Manual Spectrogram Calculation (for COM4502-6502 only)**
    
<ul>
    <li> 
        Implement a function <code>calc_SpectralPoint(xk,n)</code> which calculates a spectral point for one discrete frequency $n$ from a input frame $x[k]$, i.e. a function which implements the well-known DFT equation
    $$
    \mathrm{DFT}\{x[k]\}   =  X[n] = \frac{1}{L_{\mathrm{DFT}}} \sum \limits_{k=0}^{L_{\mathrm{DFT}}-1} x[k]  e^{j 2 \pi k n /L_{\mathrm{DFT}}}
    $$ 
    for one fixed $n$.
    </li>
    <li> 
        Implement a very similar function <code>calc_SpectralPointWindowed(xk,n)</code> which calculates a spectral point for one discrete frequency $n$ from a input frame $x[k]$, but in addition applies a window function $w[k]$ to the frame $x[k]$, i.e. the function should calculate
    $$
    \mathrm{DFT}\{w[k] \cdot x[k]\}   =  X^{\mathrm{w}}[n] = \frac{1}{L_{\mathrm{DFT}}} \sum \limits_{k=0}^{L_{\mathrm{DFT}}-1} w[k] x[k]  e^{j 2 \pi k n /L_{\mathrm{DFT}}}
    $$ 
    for one fixed $n$. The window should have the same length $L_{\mathrm{DFT}}$ as your frame and should be one of the windows we discussed during the lecture.
    </li>
    <li> 
        The functions above only calculate one spectral value at a time. To obtain a full spectrum, implement a function <code>calc_Manitude_Spectrum()</code> which transforms every windowed frame to the frequency domain and calculates all positive frequencies, i.e. for $0 \leq n \leq L_{\mathrm{DFT}}/2+1$.
    </li>
    <li> 
        Create a function <code>create_spectrogram()</code>, which splits the complete input sequence (e.g. a loaded WAVE file) into blocks of length $L_{\mathrm{DFT}}$. These may be overlapping. For each block the spectrum should be calculated using the previously implemented function <code>calc_Manitude_Spectrum()</code> and all spectra should be collected to form a spectrogram (e.g. as columns of a matrix).
    </li>
    <li> 
        Concatenate the resulting spectra to a spectrogram and display the resulting spectrogram. You can use <code>matplotlib</code>'s <code><a href="https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imshow.html">imshow()</a></code> function for manually plotting the spectrogram image. Note that a spectrogram is usually shown in dB scaling.
    </li>
    <li>
        Visualise the input signal $x[k]$ as a spectrogram for (i) a speech signal and (ii) for a chirp/sweep signal. 
    </li>
</ul>
</div>

Implement the  DFT equation
    $$\mathrm{DFT}\{x[k]\}   =  X[n] = \frac{1}{L_{\mathrm{DFT}}} \sum \limits_{k=0}^{L_{\mathrm{DFT}}-1} x[k]  e^{j 2 \pi k n /L_{\mathrm{DFT}}}$$ for one fixed $n$:

In [ ]:
def calc_SpectralPoint(xk,n):
    '''
    Implementation of the Discrete Fourier Transform (DFT).
    Calculates the Fourier coefficient X[n] for one discrete frequency n 
    
    Input: 
    xk:     time domain signal vector
    n:      discrete frequency to be calculated
    
    Output 
    Xn : discrete frequency domain point for frequency n
    '''
    
    # Your code here
    # ...

Implement DFT of windowed frame
$$\mathrm{DFT}\{w[k] \cdot x[k]\}   =  X^{\mathrm{w}}[n]  = \frac{1}{L_{\mathrm{DFT}}} \sum \limits_{k=0}^{L_{\mathrm{DFT}}-1} w[k] x[k]  e^{j 2 \pi k n /L_{\mathrm{DFT}}}$$ for one fixed $n$. 

In [ ]:
def calc_SpectralPointWindowed(xk,n,window=False):
    '''
    Implementation of the Discrete Fourier Transform (DFT).
    Calculates the Fourier coefficient X[n] for one discrete frequency n 
    
    Input: 
    xk:     time domain signal vector
    n:      discrete frequency to be calculated
    window: (optional): can be False (no window) or a window name from   
            the list of available numpy window functions, e.g. 
            np.hamming, np.bartlett, np.blackman, np.hanning, np.kaiser
            type: function
            (feel free to implement the window differently)
    
    Output 
    Xn : dicrete frequency domain point for frequency n
    '''
    # Your code here
    # L_DFT = ???
    # ...
    
    if window == False:
        win = np.ones(L_DFT) # this is a window with no effect
    else:
        None # replace this by your own window
        
    # Your code here
    # ...

The following function <code>calc_Manitude_Spectrum()</code> is supposed to transform every (windowed or not windowed) frame to the frequency domain and to calculate all positive frequencies, i.e. $X[n]$ or $X^{\mathrm{w}}[n]$ for $0 \leq n \leq L_{\mathrm{DFT}}/2+1$.

In [ ]:
def calc_Manitude_Spectrum(xk):
    '''
    Compute Fourier coefficients up to the Nyquest Limit (fs/2), i.e. Xn for n=0,...,L_DFT/2 
    using one of the two functions created before.
    and multiply the absolute value of the Fourier coefficients by 2, 
    to account for the symmetry of the Fourier coefficients above the Nyquest Limit. 
    '''
    # Your code here
    # ...
    
    # probably there should be a loop over n here

The following function <code>create_spectrogram()</code> should calculate all spectra needed for your spectrogram. 

In [ ]:
def create_spectrogram(x, L_DFT=512, noverlap):
    '''
           x: original time series
       L_DFT: The number of data points used in each block for the DFT. The default value is 512. 
    noverlap: The number of points of overlap between blocks. The default value is 256. 
    '''
    # Your code here
    # ...
    


The following function can be used to actually display the created spectrogram.

In [ ]:
def plot_spectrogram( #...

The following code actually calculates and plots your spectrogram (for the two signals mentioned above). Feel free to adapt parameters `L_DFT` and `noverlap`

In [ ]:
# load or create signal

# create and plot spectrogram (generated using your functions above)
L_DFT    = 256 # DFT length
noverlap = 84  # number of overlapping samples
starts, spec = create_spectrogram( #...
plot_spectrogram(#...

<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task 14: Apply Short-Time Fourier Transform (STFT)**
    
<ul>
    <li> 
        Explain the purpose of the Short-Time Fourier Transform (STFT) and how it differs from a regular Fourier Transform. Provide a brief(!) explanation below.
    </li>
    <li> 
        Apply the STFT to the signal generated in Task T2 using a window size of $256$ samples with $50$% overlap (COM4502-6502 students should use the previously created code (if Task T13 was completed), existing functions can be used by COM3502 students (or if COM4502-6502 students did not complete Task T13).
    </li>
    <li> 
        Interpret the spectrogram. What information does it provide about the signal? Briefly(!) explain what you see in the spectrogram and how it represents the frequency content over time.
    </li>
</ul>
</div>

In [ ]:
from scipy.signal import stft
synthetic_signal_fs = 8000
window_size = 256
overlap = 256 * 0.5
frequencies, times, Zxx = stft(audio_signal, audio_signal_fs, nperseg=window_size, noverlap=overlap)
magnitude_dB = 20 * np.log10(np.abs(Zxx))
plt.figure(figsize=(10, 6))
plt.pcolormesh(times, frequencies, magnitude_dB, shading='gouraud')
plt.title("Spectrogram of the Synthetic Signal (STFT)")
plt.ylabel("Frequency in HZ")
plt.xlabel("Time in seconds")
plt.colorbar(label='Magnitude (dB)')
plt.tight_layout()
plt.show()

**Answer to question in Task T14:**

<span style="font-weight:bold;color:orange">
T14. First Question<br><br>
The Short-Time Fourier Transform analyzes a non-stationary signal's frequency content over time by dividing it into short frames (where the signal can be approximated as stationary) and applying a window function (it can be Hanning or Hamming window) to smooth edges of each frame. The Fourier Transform is then applied to each windowed frame, providing a time-frequency representation of the signal.<br><br>

In contrast, a regular Fourier Transform provides only the frequency content of the entire signal, assuming the signal is stationary, lacks time information. STFT enables tracking frequency changes over time, which is crucial for non-stationary signals analysis.<br><br>
    
T14. Second Question<br><br>
The spectrogram presents the frequency content of the synthetic signal over time. There are three distinct, constant horizontal bands centered at 100 HZ, 300 HZ, and 500 HZ, which correspond to the three sinusoidal frequencies that we construct the synthetic signal. The absence of variation in the bands over time reflects the stationary nature of the signal, as the frequencies and phases remain unchanged throughout the 2-second duration. This indicates that the synthetic signal consists of three pure sinusoids with no noise or modulation.
</span>


## Pitch analysis

<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task 15: Pitch analysis**
    
<ul>
    <li> 
         Extract and plot the pitch contour of a speech signal over time. Write code to estimate the pitch for each frame using an autocorrelation-based pitch detection algorithm. Briefly discuss the variations in pitch. What can pitch tell us about the speaker’s speech patterns? Provide a brief analysis of the pitch contour.
    </li>
</ul>
</div>

In [ ]:
# split the speech signal into frames
speech_signal_length = len(speech_signal)   # there are 928086 samples in the speech signal
frame_duration = 0.03
frame_length = int(frame_duration * speech_signal_fs)
overlap_factor = 2   # 50% overlap between frames
overlap_length = int(np.round(frame_length/overlap_factor))
num_frames = (speech_signal_length - overlap_length) // overlap_length

# create the frame list
frames_list = []
start_sample = 0
for i in range(num_frames):
    end_sample = start_sample + frame_length
    sample_vec = speech_signal[start_sample:end_sample]
    start_sample += overlap_length
    frames_list.append(sample_vec)
print(len(frames_list))

In [ ]:
pitch_list = []
freq_min = 50  # lower frequency in HZ defining our search range
freq_max = 500  # upper frequency in HZ defining our search range
for frame_signal in frames_list:
    frame_signal_normalized = (frame_signal - np.mean(frame_signal)) / np.max(np.abs(frame_signal))
    acf = 1/len(frame_signal_normalized)*np.correlate(frame_signal_normalized, frame_signal_normalized, mode='full')
    L = len(frame_signal_normalized)
    kappa = np.arange(-(L-1), L)
    kappa_acf_center = len(acf)//2
    kappa_min = kappa_acf_center + int(np.round(fs / freq_max))
    kappa_max = kappa_acf_center + int(np.round(fs / freq_min))
    max_correlation_kappa = kappa_min + np.argmax(acf[kappa_min : kappa_max + 1])
    # calculate pitch frequency
    pitch = fs/(max_correlation_kappa-kappa_acf_center)
    pitch_list.append(pitch)

In [ ]:
total_time = ((len(frames_list) - 1) * overlap_length + frame_length) / speech_signal_fs
time = np.linspace(0, total_time, num_frames)
plt.plot(time, pitch_list)
plt.xlabel('Time in Seconds')
plt.ylabel('Frequency in HZ')
plt.title('Pitch Contour of the speech signal')

**Answer to question in Task T15:**

<span style="font-weight:bold;color:orange">
From the pitch contour diagram, we observe distinct variations in pitch over time:<br>
1. The pitch contour shows significant oscillations, indicating the speaker use varying intonations. These fluctuations may correspond to changes in emotion within the speech.<br><br>
2. Higher pitch values may represent more excited intonations, whereas lower pitch values likely correspond to calmer statements.<br><br>
3. Some regions approach zero or exhibit abrupt interruptions, likely corresponding to pauses or unvoiced segments in the speech.<br><br>

The variations in the pitch contour reflect the speaker's dynamic use of tone and rhythm in speech. These fluctuations provide insights into the speaker's emotions and sentence structures. For example, rising pitch may indicate a question, while a clamer pitch may indicate a statement. Furthermore, pitch also reveals individual vocal characteristics, such as generally lower pitch for male voices compared to female voices.
</span>

# Speech Synthesis 

<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task 16: Synthesize a simple speech sound**
    
<ul>
    <li> 
        Generate a vowel sound (e.g., “ah”) using a source-filter model, where a glottal pulse train with a frequency of 120 Hz is filtered by a vocal tract filter.
        Implement the glottal pulse train as a periodic signal and apply a simple formant-based filter. Plot the resulting waveform.
        <li> 
            Hint: Use the following formant frequencies <code>formant_freqs = [730, 1090, 2440]</code> to create filters with bandwiths of <code>bandwidths = [80, 90, 120]</code>.
        </li>
    </li>
    <li> 
        Plot the spectrogram of the synthesized sound. Compare it to the spectrogram of a real speech sample (which you can record yourself) containing the same vowel sound.
        Discuss any differences observed between synthetic and real speech.
    </li>
    <li> 
        Briefly(!) discuss any differences observed between synthetic and real speech.
    </li>
</ul>
</div>

In [ ]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

# create excitation signal function
def excitation_signal(excitation_period, length, offset=0, noise_excitation_percentage=0):
    impulse = np.zeros(length)
    impulse[offset::excitation_period] = 1  # Set impulse train
    noise = np.random.normal(size=length)
    # Normalize noise energy to match the impulse energy
    noise *= rms(impulse) / rms(noise)
    # Combine impulse and noise signals
    return (1 - noise_excitation_percentage) * impulse + noise_excitation_percentage * noise

# RMS (Root Mean Square) function
def rms(signal):
    return np.sqrt(np.mean(signal**2))

def formant_filter(signal_in, sample_rate, formant_freqs, bandwidths):
    signal_out = signal_in.copy()
    nyquist = sample_rate / 2
    for f_c, bw in zip(formant_freqs, bandwidths):
        # Normalize frequencies to Nyquist frequency
        low = (f_c - bw / 2) / nyquist  # Low cut-off normalized
        high = (f_c + bw / 2) / nyquist # High cut-off normalized
        
        # Design a Butterworth bandpass filter
        b, a = signal.butter(2, [low, high], btype='band')
        # Apply the filter to the signal
        signal_out = signal.lfilter(b, a, signal_out)
    signal_out = signal_out / np.max(np.abs(signal_out)) 
    return signal_out

In [ ]:
synthesis_signal_fs = 44100
duration = 1
length = int(synthesis_signal_fs * duration)
f0 = 120
excitation_period = synthesis_signal_fs // f0
noise_percentage = 0.1

# Generate excitation signal
excitation = excitation_signal(excitation_period, length, noise_excitation_percentage=noise_percentage)
formant_freqs = [730, 1090, 2440]
bandwidths = [80, 90, 120]
synthesis_signal = formant_filter(excitation, synthesis_signal_fs, formant_freqs, bandwidths)

In [ ]:
# Visualise the synthesis signal in time domain
synthesis_signal_time = len(synthesis_signal)/synthesis_signal_fs
synthesis_signal_t = np.linspace(0, synthesis_signal_time, len(synthesis_signal), endpoint=False)
plt.figure(figsize=(12,6*3))
plt.subplot(3, 1, 1)
plt.plot(synthesis_signal_t, synthesis_signal)
plt.xlabel('Time in second')
plt.ylabel('Amplitude')
plt.title('Time domain representation of the synthesis signal')

# Visualise the synthesis signal in frequency domain
synthesis_signal_length = len(synthesis_signal)  
# we need to find the L_DFT that is the exponential of 2 and should be larger than the length of signal
L_DFT = 2**int(np.ceil(np.log2(synthesis_signal_length)))  #2**20

# perform DFT using the FFT algorithm
spectrum = np.fft.fft(synthesis_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/synthesis_signal_fs) # the order of frequency is correspond to the output of FFT
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

# plot the positive frequency domain of the speech signal
plt.subplot(3, 1, 2)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
# plt.xlim(500, 1200)
plt.ylabel('Amplitude')
plt.title('Positive frequency domain representation of the synthesis signal')

# Visualise the synthesis signal in time-frequency 
plt.subplot(3, 1, 3)
plt.specgram(synthesis_signal, Fs=synthesis_signal_fs, NFFT=512, noverlap=256, cmap='viridis', scale='dB')
plt.title('spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')
plt.tight_layout()
plt.show()

In [ ]:
ipd.Audio(synthesis_signal, rate=synthesis_signal_fs)

In [ ]:
# read my record ah signal
ah_recording_file = 'ah_recording.wav'
ah_signal, ah_signal_fs = sf.read(ah_recording_file)
print(ah_signal_fs)

In [ ]:
ipd.Audio(ah_signal, rate=ah_signal_fs)

In [ ]:
# Visualise the ah_recording signal in time domain
ah_signal_time = len(ah_signal)/ah_signal_fs
ah_signal_t = np.linspace(0, ah_signal_time, len(ah_signal), endpoint=False)
plt.figure(figsize=(12,6*3))
plt.subplot(3, 1, 1)
plt.plot(ah_signal_t, ah_signal)
plt.xlabel('Time in second')
plt.ylabel('Amplitude')
plt.title('Time domain representation of the ah signal')

# Visualise the signal in frequency domain
ah_signal_length = len(ah_signal)  
# we need to find the L_DFT that is the exponential of 2 and should be larger than the length of signal
L_DFT = 2**int(np.ceil(np.log2(ah_signal_length)))  #2**20

# perform DFT using the FFT algorithm
spectrum = np.fft.fft(ah_signal, L_DFT)
magnitude_spectrum = np.abs(spectrum)
frequency = np.fft.fftfreq(L_DFT, d=1/ah_signal_fs) # the order of frequency is correspond to the output of FFT
positive_frequency = frequency[:L_DFT//2]
positive_magnitude_spectrum = magnitude_spectrum[:L_DFT//2]

# plot the positive frequency domain of the speech signal
plt.subplot(3, 1, 2)
plt.plot(positive_frequency, positive_magnitude_spectrum)
plt.xlabel('Frequency in HZ')
plt.ylabel('Amplitude')
plt.title('Positive frequency domain representation of the ah signal')

# Visualise the speech signal in time-frequency 
plt.subplot(3, 1, 3)
plt.specgram(ah_signal, Fs=ah_signal_fs, NFFT=512, noverlap=256, cmap='viridis', scale='dB')
plt.title('spectrogram with DFT length of 512')
plt.colorbar(label='dB')
plt.xlabel('Time in seconds')
plt.ylabel('Frequency in HZ')

**Answer to question in Task T16:**

<span style="font-weight:bold;color:orange">
   The synthesized spectrogram displays clear and static formant frequencies centered at 730 Hz, 1090 Hz, and 2440 Hz, with bandwidth deviations of 80, 90, and 120 Hz, respectively. The evenly spaced harmonics arise from the idealized glottal pulse and formant filtering process. In contrast, the real speech spectrogram reveals dynamic formant variations and broader energy distribution, particularly in the 0–5000 Hz range, with a prominent peak at the fundamental frequency of 120 Hz. The formant frequencies in real speech fluctuate slightly, observed around 700–800 Hz and 1000–1200 Hz, but the third formant appears lower, between 1900–2100 Hz, compared to the synthesized 2440 Hz. This difference may stem from slight deviations in my pronunciation of the "ah" sound. Additionally, the real speech spectrogram exhibits irregular energy above 3000 Hz, reflecting the natural articulation process and complex dynamics of vocal tract movement.
</span>

# Equaliser (for COM4502-6502 only)

We want to design an equaliser like shown in the picture below as a hardware system.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Yamaha_EQ-500_Graphic_Equalizer.jpg/1920px-Yamaha_EQ-500_Graphic_Equalizer.jpg" align="center" style="width: 500px;"/>
<center><span style="font-size:smaller">
    Picture taken from <a href="https://simple.wikipedia.org/wiki/Equalization_(audio)">Wikipedia</a>, license: <a href="https://creativecommons.org/licenses/by/2.0/">CC BY 2.0</a>
</span></center>


The following function realises one of the sliders in software.

In [ ]:
def peaking_filter(gain,center_freq,q,fs):
    """
    Derive coefficients for a peaking filter with a given amplitude and
     bandwidth.  All coefficients are calculated as described in Zölzer's
     DAFX book (ISBN: 0-471-49078-4, p. 50 - 55).  This algorithm assumes 
     a constant q-term is used through the equation.
    
    Usage:     `b,a` = peaking_filter(gain,center_freq, q,fs)
                `gain` is the logarithmic gain (in dB)
                `center_freq` is the center frequency
                `q` is q-term equating to (Fb / Fc)
                `fs` is the sampling rate
    
    Author:    Jeff Tackett 08/22/05
    Port to Python by George Close 10/07/21
    """
    
    gain = np.float32(gain)
    k = np.tan((np.pi*center_freq)/fs)
    V0 = 10**((gain)/20)
    # invert gain if a cut
    if V0 < 1:
        V0 = 1/V0

    # Boost
    if gain > 0:
        b0 = (1 + ((V0/q)*k)+ k**2) / (1+((1/q)*k)+k**2)
        b1 = (2 * (k**2 - 1)) / (1 + ((1/q)*k) + k**2);
        b2 = (1 - ((V0/q)*k) + k**2) / (1 + ((1/q)*k) + k**2);
        a1 = b1;
        a2 =  (1 - ((1/q)*k) + k**2) / (1 + ((1/q)*k) + k**2);
    # Cut
    elif gain <0:
        b0 = (1 + ((1/q)*k) + k**2) / (1 + ((V0/q)*k) + k**2);
        b1 =       (2 * (k**2 - 1)) / (1 + ((V0/q)*k) + k**2);
        b2 = (1 - ((1/q)*k) + k**2) / (1 + ((V0/q)*k) + k**2);
        a1 = b1;
        a2 = (1 - ((V0/q)*k) + k**2) / (1 + ((V0/q)*k) + k**2);
    #gain is 0
    else:
        b0 = V0;
        b1 = 0;
        b2 = 0;
        a1 = 0;
        a2 = 0;
    a = [  1, a1, a2];
    b = [ b0, b1, b2];
    return b,a

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T17: (for COM4502-6502 only)**
    
* Visualise the frequency response of one filter.
* Implement a cascade of filters to realise an equaliser.
* Visualise the frequency response of your equaliser filter and the input and (filtered) output signal.
    
</div>

In [ ]:
# Your code here 
#
# ...

# Prepare for submission

<br>
<div style="border: 2px solid #999; padding: 10px; background: #abe;">
    
**Task T18:**
 
* Clear all cell outputs to reduce the file size (in Jupyter Notebooks click on "Cell->All Output->Clear")
* Create a `.zip` file named `YourName.zip` containing this Jupyter Notebook files as well as all other files necessary to run this notebook (**if such exist**, e.g. if you created (additional) WAVE files). 
* Hand in your `.zip` file via Blackboard.
    

<span style="font-weight:bold;color:red;text-align:center;">**Important: For marking, we expect your code to work ‘out of the box’.**</span> This means that no additional software should need to be installed to make the Notebook run. If you only used libraries known from the Speech Processing Lab classes, you should be safe here.
    
</div>